In [1]:
###Made by Khalil Droubi
###To process Iolite baseline-subtracted NP-II files for BB

To Do:

-Do a Final Check if MSWD calculations are correct in AB_err function

-Write function to chop lines up into smaller intervals, to test how long of a line we really need.

-Make Age calculation work

In [2]:
import pandas as pd
import os
import re
import copy
import numpy as np
import math
from scipy import stats
from scipy.stats import linregress

#Graphing stuff
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

# %pip install seaborn
# import seaborn as sns

#%pip install PyPDF2
from PyPDF2 import PdfFileMerger, PdfFileReader

#%pip install pdfkit
import pdfkit

#%pip install xlsxwriter
import xlsxwriter

#pd.set_option("display.precision", 8)

In [3]:
def calc207_206(work_age):
    
    decay_238 = 1.55125E-10
    decay_235 = 9.8485E-10
    decay_232 = 4.9475E-11
    
    working_207_206 = (1/137.818)*(math.exp(decay_238*work_age)-1) / (math.exp(decay_235*work_age) -1)
    return working_207_206

def calc206_238(work_age):
    
    decay_238 = 1.55125E-10
    
    ratio = (math.exp(decay_238 * work_age)) -1
    
    return ratio

def age207vs206_calc(input207_206, age_guess = 1*(10**9), years_multiplier = 2):
    work_age = age_guess
    
    
    working_207_206 = calc207_206(work_age)

    diff = input207_206 - working_207_206

    if diff > 0:
        #print('debug')
        while abs(diff) > 0.00000000000000001:
            #print('debug')
            work_age = work_age + 1*(10**years_multiplier)
            working_207_206 = calc207_206(work_age)
            diff = input207_206 - working_207_206
    if diff < 0:
        while abs(diff) > 0.00000000000000001:
            work_age = work_age - 1*(10**years_multiplier)
            working_207_206 = calc207_206(work_age)
            diff = input207_206 - working_207_206
            
    print('Succesful calculation!?')        
    return work_age

def age206vs238_calc(input206_238, age_guess = 1*(10**9), years_multiplier = 2):
    work_age = age_guess
    
    
    working_206_238 = calc206_238(work_age)

    diff = input206_238 - working_206_238

    if diff > 0:
        #print('debug')
        while abs(diff) > 0.00000000000000001:
          
            work_age = work_age + 1*(10**years_multiplier)
            working_206_238 = calc206_238(work_age)
            diff = input206_238 - working_206_238
    if diff < 0:
        while abs(diff) > 0.00000000000000001:
            
            work_age = work_age - 1*(10**years_multiplier)
            working_207_206 = calc206_238(work_age)
            diff = input206_238 - working_206_238
            
            
    return work_age


In [4]:
#Functions for calculation and EXCEL export

def read_np2_timeseries(excel_file):
    ''' Excel input file is your baseline corrected time series export from Iolite for the NP-II.'''
    df = pd.read_excel(excel_file, sheet_name = None)
    keys = df.keys()
    header_row = 0
    new_dict = {}
    for key in keys:
        if '.' in key: #Kind of hard-coded right now, so if names get weird may need to change
            
            df_test = df[key]

            df_test.columns = df_test.iloc[header_row]
            df_test = df_test.drop(header_row)
            df_test = df_test.reset_index(drop=True)
            
#             test1_new = df_test[['Absolute Time',
#              'Elapsed Time',
#                  'm238_CPS',
#                  'm232_CPS',
#                  'm208_CPS',
#                  'm207_CPS',
#                  'm206_CPS',
#                  'm204_CPS',
#                  'm202_CPS',]]
            
            new_string = key.split('time')[0].rstrip()
            new_dict[new_string] = df_test #test1_new
    return new_dict

def calc_CPS(np2_dict):
    columns = ['Absolute Time',
     'Elapsed Time',
     'm238_CPS',
     'm232_CPS',
     'm208_CPS',
     'm207_CPS',
     'm206_CPS',
     'm204_CPS',
     'm202_CPS']

    new_col = ['Absolute Time',
     'Elapsed Time',
     '238_CPS',
     '232_CPS',
     '208_CPS',
     '207_CPS',
     '206_CPS',
     '204_CPS',
     '202_CPS']

    cut_col = ['238_CPS',
     '232_CPS',
     '208_CPS',
     '207_CPS',
     '206_CPS',
     '204_CPS',
     '202_CPS']

    calc_dict = {}
    for key in np2_dict:
        #print(key)
        test_df1 = np2_dict[key]

        for col in columns:
                test_df2 = test_df1.apply(lambda x: x * 62500000 if 'CPS' in x.name else x)
                test_df2 = test_df2[['Absolute Time',
             'Elapsed Time',
                 'm238_CPS',
                 'm232_CPS',
                 'm208_CPS',
                 'm207_CPS',
                 'm206_CPS',
                 'm204_CPS',
                 'm202_CPS',]]
        test_df2.columns = new_col
        test_df2 = test_df2[cut_col]
        result = pd.concat([test_df1, test_df2], axis=1)
        
         #Calculating OPZ
        result['OPZ_238'] = result.apply(lambda x: x['m238'] - x['m238_CPS'], axis=1)
        result['OPZ_232'] = result.apply(lambda x: x['m232'] - x['m232_CPS'], axis=1)
        result['OPZ_208'] = result.apply(lambda x: x['m208'] - x['m208_CPS'], axis=1)
        result['OPZ_207'] = result.apply(lambda x: x['m207'] - x['m207_CPS'], axis=1)
        result['OPZ_206'] = result.apply(lambda x: x['m206'] - x['m206_CPS'], axis=1)
        result['OPZ_204'] = result.apply(lambda x: x['m204'] - x['m204_CPS'], axis=1)
        result['OPZ_202'] = result.apply(lambda x: x['m202'] - x['m202_CPS'], axis=1)
        result['OPZ_208/206'] = result.apply(lambda x: x['OPZ_208'] / x['OPZ_206'], axis=1)
        result['OPZ_207/206'] = result.apply(lambda x: x['OPZ_207'] / x['OPZ_206'], axis=1)
        result['OPZ_206/204_Hg-corrected'] = result.apply(lambda x: x['OPZ_206'] / (x['OPZ_204'] - (x['OPZ_202'] * 6.87/29.86)) , axis=1)
        
        
       
        #Calculating Signal-to-Noise Ratios [V]/[V]
        result['SNR_238'] = result.apply(lambda x: x['m238_CPS']/ x['OPZ_238'], axis=1)
        result['SNR_232'] = result.apply(lambda x: x['m232_CPS']/ x['OPZ_232'], axis=1)
        result['SNR_208'] = result.apply(lambda x: x['m208_CPS']/ x['OPZ_208'], axis=1)
        result['SNR_207'] = result.apply(lambda x: x['m207_CPS']/ x['OPZ_207'], axis=1)
        result['SNR_206'] = result.apply(lambda x: x['m206_CPS']/ x['OPZ_206'], axis=1)
        result['SNR_204'] = result.apply(lambda x: x['m204_CPS']/ x['OPZ_204'], axis=1)
        result['SNR_202'] = result.apply(lambda x: x['m202_CPS']/ x['OPZ_202'], axis=1)
        
        #Calculating Ratios
        result['206/238'] = result.apply(lambda x: x['206_CPS']/x['238_CPS'], axis=1)
        result['208/232'] = result.apply(lambda x: x['208_CPS']/x['232_CPS'], axis=1)
        result['207/206'] = result.apply(lambda x: x['207_CPS']/x['206_CPS'], axis=1)
        result['208/206'] = result.apply(lambda x: x['208_CPS']/x['206_CPS'], axis=1)
        result['206/204'] = result.apply(lambda x: x['206_CPS']/x['204_CPS'], axis=1)
        result['208/204'] = result.apply(lambda x: x['208_CPS']/x['204_CPS'], axis=1)
        result['207/204'] = result.apply(lambda x: x['207_CPS']/x['204_CPS'], axis=1)
        
        calc_dict[key] = result
    
    return calc_dict

def ranked_minimization(sheet, ratio, reject_percentage = 20):

    mytest = tester[sheet].copy(deep=True)

    df_mean_before = mytest[ratio].mean()
    df_1std_before = mytest[ratio].std()
    df_count_before = mytest[ratio].count()
    df_2se_perc_before = (2 * mytest[ratio].sem()) / df_mean_before * 100

    dif_mean = ratio + '_dif_from_mean'
    dif_1SD = ratio + '_dif_from_1SD'
    mytest[dif_mean] = mytest.apply(lambda x: abs(x[ratio] - df_mean_before), axis=1)
    mytest[dif_1SD] = mytest.apply(lambda x: x[dif_mean] - df_1std_before, axis=1)


    mytest2 = mytest.sort_values(by = dif_1SD, ascending = False)
    #mytest2.head()

    ratios_to_reject = int(mytest[ratio].count() * reject_percentage / 100)
    #print(ratios_to_reject)

    after_rejection = mytest2[ratios_to_reject:]

    df_mean_after = after_rejection[ratio].mean()
    df_1std_after = after_rejection[ratio].std()
    df_count_after = after_rejection[ratio].count()
    
    #This is not 2SE%, should probably fix labels... KD 14 June 2021
    df_2se_perc_after = (2 * after_rejection[ratio].std()) / df_mean_after * 100

    # print(df_mean_after)
    # print(df_1std_after)
    # print(df_2se_perc_after)

    results_dict = {}
    
    results_dict['avg_before'] = df_mean_before
    results_dict['1sd_before'] = df_1std_before
    results_dict['2se%_before'] = df_2se_perc_before
    results_dict['avg_after'] = df_mean_after
    results_dict['1sd_after'] = df_1std_after
    results_dict['2σ%_after'] = df_2se_perc_after
    
    return results_dict

def statistics_NP2(calc_dict):
    calc_list = ['238_CPS', '232_CPS',
           '208_CPS', '207_CPS', '206_CPS', '204_CPS', '202_CPS', '206/238',
           '208/232', '207/206', '208/206', '206/204','208/204','207/204'  ]
    mega_dict = {}

    for sheet in calc_dict:
        tester = calc_dict[sheet]
        stats_dict = {}
        for col in tester:

            if col in calc_list:
                #print(col)
                if '/' in col:
                    key = col + '_before rejection'
                else:
                    key = col + '_mean'
                df_mean = tester[col].mean()
                stats_dict[key] = df_mean
                #This is not 2SE%, should probably fix labels... KD 14 June 2021
                df_precision = (2 * tester[col].std()) / df_mean * 100
                stats_dict[col + '_2σ%'] = df_precision
            if 'OPZ' in col:
                 stats_dict[col + '_mean'] = tester[col].mean()
            if 'SNR' in col:
                 stats_dict[col + '_mean'] = tester[col].mean() 
                    
        stats_dict['Time (s)'] = tester['Elapsed Time'].max()
        
        #new_string = sheet.replace('time series data', '')
        new_string = sheet.split('time')[0].rstrip()
        mega_dict[new_string] = stats_dict

    df_1 = pd.DataFrame(mega_dict)
    df_flip = pd.DataFrame.transpose(df_1)
    return df_flip

def statistics_ranktest(calc_dict, reject_percentage = 20):
    calc_list = ['238_CPS', '232_CPS',
           '208_CPS', '207_CPS', '206_CPS', '204_CPS', '202_CPS', '206/238',
           '208/232', '207/206', '208/206', '206/204','208/204','207/204'  ]
    mega_dict = {}

    for sheet in calc_dict:
        tester = calc_dict[sheet]
        stats_dict = {}
        for col in tester:

            if col in calc_list:
                #print(col)
                if '/' in col:
                    key_bf = col + '_before rejection'
                    key_af = col + '_after rejection'
                    key_bf_se = col + '_before rejection 2se%'
                    key_af_se = col + '_after rejection 2σ%'
                    
                    ranked_dict = ranked_minimization(sheet, col, reject_percentage)
                    stats_dict[key_bf] = ranked_dict['avg_before']
                    stats_dict[key_bf_se] = ranked_dict['2se%_before']
                    stats_dict[key_af] = ranked_dict['avg_after']
                    #This is not 2SE%, should probably fix labels... KD 14 June 2021
                    stats_dict[key_af_se] = ranked_dict['2σ%_after']
                else:
                    key = col + '_mean'
                    df_mean = tester[col].mean()
                    stats_dict[key] = df_mean
                    #This is not 2SE%, should probably fix labels... KD 14 June 2021
                    df_precision = (2 * tester[col].std()) / df_mean * 100
                    stats_dict[col + '_2σ%'] = df_precision
            if 'OPZ' in col:
                 stats_dict[col + '_mean'] = tester[col].mean()
            if 'SNR' in col:
                 stats_dict[col + '_mean'] = tester[col].mean()
                    
        stats_dict['Time (s)'] = tester['Elapsed Time'].max()
        
        #new_string = sheet.replace('time series data', '')
        new_string = sheet.split('time')[0].rstrip()
        mega_dict[new_string] = stats_dict

    df_1 = pd.DataFrame(mega_dict)
    df_flip = pd.DataFrame.transpose(df_1)
    return df_flip

### Getting rid of negatives

def ranked_minimization2(sheet, ratio, reject_percentage = 20):
    ''' Modified to exclude negative ratios BEFORE rejection.'''
    
    mytest = tester[sheet].copy(deep=True)

    df_mean_before = mytest[ratio].mean()
    df_1std_before = mytest[ratio].std()
    df_count_before = mytest[ratio].count()
    df_2se_perc_before = (2 * mytest[ratio].sem()) / df_mean_before * 100

    dif_mean = ratio + '_dif_from_mean'
    dif_1SD = ratio + '_dif_from_1SD'
    mytest[dif_mean] = mytest.apply(lambda x: abs(x[ratio] - df_mean_before), axis=1)
    mytest[dif_1SD] = mytest.apply(lambda x: x[dif_mean] - df_1std_before, axis=1)

    mytest_noNeg = mytest[mytest[ratio] > 0]
    
    mytest2 = mytest_noNeg.sort_values(by = dif_1SD, ascending = False)
    #mytest2.head()

    ratios_to_reject = int(mytest_noNeg[ratio].count() * reject_percentage / 100)
    #print(ratios_to_reject)

    after_rejection = mytest2[ratios_to_reject:]
    
    

    df_mean_after = after_rejection[ratio].mean()
    df_1std_after = after_rejection[ratio].std()
    df_count_after = after_rejection[ratio].count()
   
    df_2se_perc_after = (2 * after_rejection[ratio].sem()) / df_mean_after * 100
    df_2sd_perc_after = (2 * after_rejection[ratio].std()) / df_mean_after * 100
    

    # print(df_mean_after)
    # print(df_1std_after)
    # print(df_2se_perc_after)

    results_dict = {}
    
    results_dict['avg_before'] = df_mean_before
    results_dict['1sd_before'] = df_1std_before
    results_dict['2se%_before'] = df_2se_perc_before
    results_dict['avg_after'] = df_mean_after
    results_dict['1sd_after'] = df_1std_after
    results_dict['2se%_after'] = df_2se_perc_after
    results_dict['2σ%_after'] = df_2sd_perc_after
    
    
    return results_dict

def statistics_ranktest2(calc_dict, reject_percentage = 20):
    '''Incorporates the new ranked minimization.'''
    calc_list = ['238_CPS', '232_CPS',
           '208_CPS', '207_CPS', '206_CPS', '204_CPS', '202_CPS', '206/238',
           '208/232', '207/206', '208/206', '206/204','208/204','207/204'  ]
    mega_dict = {}

    for sheet in calc_dict:
        tester = calc_dict[sheet]
        stats_dict = {}
        for col in tester:

            if col in calc_list:
                #print(col)
                if '/' in col:
                    key_bf = col + '_before rejection'
                    key_af = col + '_after rejection'
                    key_bf_se = col + '_before rejection 2se%'
                    key_af_se = col + '_after rejection 2se%'
                    key_af_2sd = col + '_after rejection 2σ%'
                    
                    ranked_dict = ranked_minimization2(sheet, col, reject_percentage)
                    stats_dict[key_bf] = ranked_dict['avg_before']
                    stats_dict[key_bf_se] = ranked_dict['2se%_before']
                    stats_dict[key_af] = ranked_dict['avg_after']
                    
                    stats_dict[key_af_se] = ranked_dict['2se%_after']
                    stats_dict[key_af_2sd] = ranked_dict['2σ%_after']
                else:
                    key = col + '_mean'
                    df_mean = tester[col].mean()
                    stats_dict[key] = df_mean
                    #This is not 2SE%, should probably fix labels... KD 14 June 2021
                    df_precision = (2 * tester[col].std()) / df_mean * 100
                    stats_dict[col + '_2σ%'] = df_precision
            if 'OPZ' in col:
                 stats_dict[col + '_mean'] = tester[col].mean()
            if 'SNR' in col:
                 stats_dict[col + '_mean'] = tester[col].mean()
                
        stats_dict['Time (s)'] = tester['Elapsed Time'].max()
        
        #new_string = sheet.replace('time series data', '')
        new_string = sheet.split('time')[0].rstrip()
        mega_dict[new_string] = stats_dict

    df_1 = pd.DataFrame(mega_dict)
    df_flip = pd.DataFrame.transpose(df_1)
    return df_flip

def stat_grouper(stats_df):
    ''' Input dataframe that was output from statistics_ranktest2 function.'''
    headers_to_keep = [
        '206/238_after rejection',
        '206/238_after rejection 2se%',
        '208/232_after rejection',
        '208/232_after rejection 2se%',
        '207/206_after rejection',
        '207/206_after rejection 2se%',
        '208/206_after rejection',
        '208/206_after rejection 2se%',
        '206/204_after rejection',
        '206/204_after rejection 2se%',
        '208/204_after rejection',
        '208/204_after rejection 2se%',
        '207/204_after rejection',
        '207/204_after rejection 2se%'   
    ]

    headers_to_math = [
        '206/238_after rejection',   
        '208/232_after rejection',  
        '207/206_after rejection',    
        '208/206_after rejection',    
        '206/204_after rejection',    
        '208/204_after rejection',   
        '207/204_after rejection',
    ]


    short_tester = stats_df[headers_to_keep]

    samples = short_tester.index.values.tolist()
    group = None
    group_names = []
    start = [0]
    end = []
    for idx in range(len(samples)):
        if group == None:
            group = samples[idx].split(' 1')[0]
            group_names.append(group)
        #print(samples[idx])
        if group in samples[idx]:
            pass
        else:
            end.append(idx)
            start.append(idx + 1)
            name = samples[idx].split(' 1')[0]
            group = name
            group_names.append(name)
    end.append(len(samples))

    # print('start:', start)
    # print('end:', end)
    # print('group names:', group_names)

    stats_dict = {}
    for idx in range(len(group_names)):
        group_dict = {}
        #print(idx)
        group_df = short_tester.iloc[start[idx]:end[idx]]

        for col in headers_to_math:
            #print(group_df[col])
            name = col.split('_')[0]
            df_mean = group_df[col].mean()
            #print(df_mean)
            group_dict[name + '_mean'] = df_mean
            df_2se_perc = (2 * group_df[col].std())/df_mean * 100
            group_dict[name + '_2SE%'] = df_2se_perc 

        stats_dict[group_names[idx]] = group_dict

    return pd.DataFrame(stats_dict)

def stat_correcter2(stats_df, std = 'glass'):
    ''' Input dataframe that was output from statistics_ranktest2
    function.'''
    headers_to_keep = [
        '206/238_after rejection',
        '206/238_after rejection 2se%',
        '206/238_after rejection 2σ%',
        '208/232_after rejection',
        '208/232_after rejection 2se%',
        '208/232_after rejection 2σ%',
        '207/206_after rejection',
        '207/206_after rejection 2se%',
        '207/206_after rejection 2σ%',
        '208/206_after rejection',
        '208/206_after rejection 2se%',
        '208/206_after rejection 2σ%',
        '206/204_after rejection',
        '206/204_after rejection 2se%',
        '206/204_after rejection 2σ%',
        '208/204_after rejection',
        '208/204_after rejection 2se%',
        '208/204_after rejection 2σ%',
        '207/204_after rejection',
        '207/204_after rejection 2se%',
        '207/204_after rejection 2σ%',
    ]

    headers_to_math = [
        '206/238_after rejection',   
        '208/232_after rejection',  
        '207/206_after rejection',    
        '208/206_after rejection',    
        '206/204_after rejection',    
        '208/204_after rejection',   
        '207/204_after rejection',
    ]
    
    ### Correction Math
    
    #Currently hard-coded, will need to define in function at some point...
    
    published_ratios_610 = {
        '206/238': 0.258174418887103,
        '208/232': 0.546910763260703,
        '207/206': 0.909778846717897,
        '208/206': 2.16900334369684,
        '206/204': 17.047,
        '207/204': 15.509,
        '208/204': 36.975,
        
    }
    published_ratios_MKED = {
        '206/238': 0.26538,
        '208/232': 0.08127,
        '207/206': 0.09465,
        '208/206': 0.8719,
        '206/204': 18962.2,
        '207/204': 1794.77223,
        '208/204': 16533.14218,      
    }
    published_ratios_612 = {
        '206/238': 0.289090155580635,
        '208/232': 0.598868250924868,
        '207/206': 0.907335907335907,
        '208/206': 2.16450216450216,
        '206/204': 17.094,
        '207/204': 15.51,
        '208/204': 37,      
    }
    published_ratios_BHVO = {
        '206/238': 1.24298582359808,
        '208/232': 0.811596334965975,
        '207/206': 0.831206960977953,
        '208/206': 2.042918913147926,
        '206/204': 18.733,
        '207/204': 15.571,
        '208/204': 38.27,      
    }   
    
    
    if std == 'glass':
        #print('610 primary std')
        published_ratios = published_ratios_610
    elif std == '612':
        #print('612 primary std')
        published_ratios = published_ratios_612
    elif std == 'BHVO':
        #print('BHVO primary std')
        published_ratios = published_ratios_BHVO    
    else:
        published_ratios = published_ratios_MKED 
    
    short_tester = stats_df[headers_to_keep]
    result = short_tester.copy(deep=True)

    samples = short_tester.index.values.tolist()
    group = None
    group_names = []
    start = [0]
    end = []
    for idx in range(len(samples)):
        if group == None:
            group = samples[idx].split(' 1')[0]
            group_names.append(group)
        #print(samples[idx])
        if group in samples[idx]:
            pass
        else:
            end.append(idx)
           
            start.append(idx)
            name = samples[idx].split(' 1')[0]
            group = name
            group_names.append(name)
    end.append(len(samples))

    # print('start:', start)
    # print('end:', end)
    # print('group names:', group_names)

    stats_dict = {}
    corrected_stats_dict = {}
    for idx in range(len(group_names)):
        group_dict = {}
        #print(idx)
        group_df = short_tester.iloc[start[idx]:end[idx]]
        #print(group_df.index.values.tolist())
        for col in headers_to_math:
            #print(group_df[col])
            name = col.split('_')[0]
            df_mean = group_df[col].mean()
            #print(df_mean)
            group_dict[name + '_mean'] = df_mean
            #External error
            df_2sd_perc = (2 * group_df[col].std())/df_mean * 100
            
            group_dict[name + '_2\u03C3%'] = df_2sd_perc
            
            df_2se_perc = (2 * group_df[col].sem())/df_mean * 100
            
            group_dict[name + '_2se%'] = df_2se_perc
            
   

        stats_dict[group_names[idx]] = group_dict
    
    grouper = pd.DataFrame(stats_dict)
    #grouper['SRM NIST 610']['206/238_mean']
    if std == 'glass':
        #print('610 primary std')
        primary_std = grouper['SRM NIST 610']
    elif std == '612':
        #print('612 primary std')
        primary_std = grouper['SRM NIST 612']
    elif std == 'BHVO':
        #print('BHVO primary std')
        primary_std = grouper['BHVO-2G']         
    else:
        primary_std = grouper['MKED']
        
    

        #Applying correction Hard coding the primary standard
    result['206/238 corrected'] = short_tester.apply(lambda x: x['206/238_after rejection']/ (primary_std['206/238_mean'] / published_ratios['206/238']), axis=1)
    result['208/232 corrected'] = short_tester.apply(lambda x: x['208/232_after rejection']/ (primary_std['208/232_mean'] / published_ratios['208/232']), axis=1)
    result['207/206 corrected'] = short_tester.apply(lambda x: x['207/206_after rejection']/ (primary_std['207/206_mean'] / published_ratios['207/206']), axis=1)
    result['208/206 corrected'] = short_tester.apply(lambda x: x['208/206_after rejection']/(primary_std['208/206_mean'] / published_ratios['208/206']), axis=1)
    result['206/204 corrected'] = short_tester.apply(lambda x: x['206/204_after rejection']/(primary_std['206/204_mean'] / published_ratios['206/204']), axis=1)
    result['207/204 corrected'] = short_tester.apply(lambda x: x['207/204_after rejection']/(primary_std['207/204_mean'] / published_ratios['207/204']), axis=1)
    result['208/204 corrected'] = short_tester.apply(lambda x: x['208/204_after rejection']/(primary_std['208/204_mean'] / published_ratios['208/204']), axis=1)

#     #Making 2se for plotting in EXCEL
    
    result['206/238_after rejection 2se'] = short_tester.apply(lambda x: x['206/238_after rejection'] * x['206/238_after rejection 2se%'] / 100, axis=1)
    result['208/232_after rejection 2se'] = short_tester.apply(lambda x: x['208/232_after rejection'] * x['208/232_after rejection 2se%'] / 100, axis=1)
    result['207/206_after rejection 2se'] = short_tester.apply(lambda x: x['207/206_after rejection'] * x['207/206_after rejection 2se%'] / 100, axis=1)
    result['208/206_after rejection 2se'] = short_tester.apply(lambda x: x['208/206_after rejection'] * x['208/206_after rejection 2se%'] / 100, axis=1)
    result['206/204_after rejection 2se'] = short_tester.apply(lambda x: x['206/204_after rejection'] * x['206/204_after rejection 2se%'] / 100, axis=1)
    result['207/204_after rejection 2se'] = short_tester.apply(lambda x: x['207/204_after rejection'] * x['207/204_after rejection 2se%'] / 100, axis=1)
    result['208/204_after rejection 2se'] = short_tester.apply(lambda x: x['208/204_after rejection'] * x['208/204_after rejection 2se%'] / 100, axis=1)
    
    
    #Age calculation
    
#     result['206/238 Age [Ma]'] = result.apply(lambda x: age206vs238_calc(x['206/238 corrected'], 500*(10**9), 6) / 1000000, axis=1)
#     result['207/206 Age [Ma]'] = result.apply(lambda x: age207vs206_calc(x['207/206 corrected'], 500*(10**9), 6) / 1000000, axis=1)
    
    
    result_headers = [
        '206/238 corrected',
        '208/232 corrected',
        '207/206 corrected',
        '208/206 corrected',
        '206/204 corrected',
        '207/204 corrected',
        '208/204 corrected',
#         '206/238 Age [Ma]',
#         '207/206 Age [Ma]',
    ]
       
    
    for idx in range(len(group_names)):
        group_dict = {}
        #print(idx)
        group_df = result.iloc[start[idx]:end[idx]]
        #print(group_df.index.values.tolist())
        #print(group_df)
        for col in result_headers:
            #print(group_df[col])
            name = col.split('_')[0]
            df_mean = group_df[col].mean()
            #print(df_mean)
            group_dict[name + '_mean'] = df_mean
            df_2sd_perc = (2 * group_df[col].std())/df_mean * 100
            group_dict[name + '_2\u03C3%'] = df_2sd_perc 
            df_2sd = 2 * group_df[col].std()
            group_dict[name + '_2\u03C3'] = df_2sd
            
            df_2se_perc = (2 * group_df[col].sem())/df_mean * 100
            group_dict[name + '_2se%'] = df_2se_perc
            df_2se = 2 * group_df[col].sem()
            group_dict[name + '_2se'] = df_2se
        
        

        corrected_stats_dict[group_names[idx]] = group_dict
  
   
    corrected_grouper = pd.DataFrame(corrected_stats_dict)
   
   
    
    grouper_flip = pd.DataFrame.transpose(grouper)
    corrected_flip = pd.DataFrame.transpose(corrected_grouper)
    grouper_list = [grouper_flip, corrected_flip]
    
    grouper_comb = pd.concat(grouper_list, axis=1)
    
    #### Propogating Errors
    
    if std == 'glass':
        print('610 primary std')
        external_err_std = grouper_comb.loc['SRM NIST 610']
    elif std == '612':
        print('612 primary std')
        external_err_std = grouper_comb.loc['SRM NIST 612']
    elif std == 'BHVO':
        print('BHVO primary std')
        external_err_std = grouper_comb.loc['BHVO-2G']    
    else:
        print('MKED primary std')
        external_err_std = grouper_comb.loc['MKED']
        
    
    
    
    result['206/238 corrected BB error%'] = short_tester.apply(lambda x: ((x['206/238_after rejection 2se%'])**2 + (external_err_std['206/238 corrected_2\u03C3%'])**2)**0.5, axis=1)
    result['208/232 corrected BB error%'] = short_tester.apply(lambda x: ((x['208/232_after rejection 2se%'])**2 + (external_err_std['208/232 corrected_2\u03C3%'])**2)**0.5, axis=1)
    result['207/206 corrected BB error%'] = short_tester.apply(lambda x: ((x['207/206_after rejection 2se%'])**2 + (external_err_std['207/206 corrected_2\u03C3%'])**2)**0.5, axis=1)
    result['208/206 corrected BB error%'] = short_tester.apply(lambda x: ((x['208/206_after rejection 2se%'])**2 + (external_err_std['208/206 corrected_2\u03C3%'])**2)**0.5, axis=1)
    result['206/204 corrected BB error%'] = short_tester.apply(lambda x: ((x['206/204_after rejection 2se%'])**2 + (external_err_std['206/204 corrected_2\u03C3%'])**2)**0.5, axis=1)
    result['207/204 corrected BB error%'] = short_tester.apply(lambda x: ((x['207/204_after rejection 2se%'])**2 + (external_err_std['207/204 corrected_2\u03C3%'])**2)**0.5, axis=1)
    result['208/204 corrected BB error%'] = short_tester.apply(lambda x: ((x['208/204_after rejection 2se%'])**2 + (external_err_std['208/204 corrected_2\u03C3%'])**2)**0.5, axis=1)
 
    result['206/238 corrected BB error'] = result.apply(lambda x: (x['206/238 corrected BB error%'])* x['206/238 corrected']/100, axis=1)
    result['208/232 corrected BB error'] = result.apply(lambda x: (x['208/232 corrected BB error%'])* x['208/232 corrected']/100, axis=1)
    result['207/206 corrected BB error'] = result.apply(lambda x: (x['207/206 corrected BB error%'])* x['207/206 corrected']/100, axis=1) 
    result['208/206 corrected BB error'] = result.apply(lambda x: (x['208/206 corrected BB error%'])* x['208/206 corrected']/100, axis=1)
    result['206/204 corrected BB error'] = result.apply(lambda x: (x['206/204 corrected BB error%'])* x['206/204 corrected']/100, axis=1)   
    result['207/204 corrected BB error'] = result.apply(lambda x: (x['207/204 corrected BB error%'])* x['207/204 corrected']/100, axis=1)
    result['208/204 corrected BB error'] = result.apply(lambda x: (x['208/204 corrected BB error%'])* x['208/204 corrected']/100, axis=1)
    
   
    
    new_headers = [
        '206/238 corrected BB error%',
        '208/232 corrected BB error%',
        '207/206 corrected BB error%',
        '208/206 corrected BB error%',
        '206/204 corrected BB error%',
        '207/204 corrected BB error%',
        '208/204 corrected BB error%',
        '206/238 corrected BB error',
        '208/232 corrected BB error',
        '207/206 corrected BB error',
        '208/206 corrected BB error',
        '206/204 corrected BB error',
        '207/204 corrected BB error',
        '208/204 corrected BB error'    
    ]
    
    new_stats_dict = {}
    for idx in range(len(group_names)):
        group_dict = {}
        #print(idx)
        group_df = result.iloc[start[idx]:end[idx]]
        #print(group_df)
        for col in new_headers:
            #print(group_df[col])
            #name = col.split('_')[0]
            
            df_mean = group_df[col].mean()
            
            #print(df_mean)
            group_dict[col + '_mean'] = df_mean
        
        new_stats_dict[group_names[idx]] = group_dict 
            
    extra_grouper = pd.DataFrame(new_stats_dict)
    
   
    corrected_flip2 = pd.DataFrame.transpose(extra_grouper)
    grouper_list2 = [grouper_comb, corrected_flip2]
    
    grouper_comb_final = pd.concat(grouper_list2, axis=1)
    #print('GROUPER',grouper_comb_final.keys())
    #print('RESULT', result.keys())
    ############

    
    grouper_order = ['206/238_mean', '206/238_2σ%','206/238_2se%', '208/232_mean', '208/232_2σ%','208/232_2se%',
       '207/206_mean', '207/206_2σ%','207/206_2se%', '208/206_mean', '208/206_2σ%', '208/206_2se%',
       '206/204_mean', '206/204_2σ%','206/204_2se%', '208/204_mean', '208/204_2σ%','208/204_2se%',
       '207/204_mean', '207/204_2σ%', '207/204_2se%',
        '206/238 corrected_mean','206/238 corrected_2σ','206/238 corrected_2se%',
       '206/238 corrected_2se',
       '206/238 corrected_2σ%', '206/238 corrected BB error_mean', '206/238 corrected BB error%_mean',
       '208/232 corrected_mean','208/232 corrected_2σ', '208/232 corrected_2σ%', '208/232 corrected_2se%', '208/232 corrected_2se','208/232 corrected BB error_mean', '208/232 corrected BB error%_mean',
       '207/206 corrected_mean', '207/206 corrected_2σ', '207/206 corrected_2σ%','207/206 corrected_2se%',
       '207/206 corrected_2se', '207/206 corrected BB error_mean','207/206 corrected BB error%_mean',
        '208/206 corrected_mean', '208/206 corrected_2σ','208/206 corrected_2σ%', '208/206 corrected_2se%', '208/206 corrected_2se','208/206 corrected BB error_mean', '208/206 corrected BB error%_mean',
        '206/204 corrected_mean','206/204 corrected_2σ','206/204 corrected_2σ%', '206/204 corrected_2se%',
       '206/204 corrected_2se', '206/204 corrected BB error_mean', '206/204 corrected BB error%_mean',
      '207/204 corrected_mean','207/204 corrected_2σ', '207/204 corrected_2σ%', '207/204 corrected_2se%', '207/204 corrected_2se', '207/204 corrected BB error_mean', '207/204 corrected BB error%_mean',
      '208/204 corrected_mean', '208/204 corrected_2σ','208/204 corrected_2σ%','208/204 corrected_2se%',
       '208/204 corrected_2se', '208/204 corrected BB error_mean', '208/204 corrected BB error%_mean', 
        #'206/238 Age [Ma]_mean','207/206 Age [Ma]_mean', 
        ]
    
    result_order = ['206/238_after rejection','206/238_after rejection 2σ%', '206/238_after rejection 2se%','206/238_after rejection 2se',
       '208/232_after rejection','208/232_after rejection 2σ%', '208/232_after rejection 2se%','208/232_after rejection 2se',
       '207/206_after rejection','207/206_after rejection 2σ%', '207/206_after rejection 2se%','207/206_after rejection 2se',
       '208/206_after rejection','208/206_after rejection 2σ%', '208/206_after rejection 2se%','208/206_after rejection 2se',
       '206/204_after rejection','206/204_after rejection 2σ%', '206/204_after rejection 2se%','206/204_after rejection 2se',
       '208/204_after rejection','208/204_after rejection 2σ%', '208/204_after rejection 2se%','208/204_after rejection 2se',
       '207/204_after rejection','207/204_after rejection 2σ%', '207/204_after rejection 2se%','207/204_after rejection 2se',
       '206/238 corrected', '206/238 corrected BB error', '206/238 corrected BB error%',
       '208/232 corrected', '208/232 corrected BB error','208/232 corrected BB error%',
        '207/206 corrected', '207/206 corrected BB error', '207/206 corrected BB error%',
       '208/206 corrected', '208/206 corrected BB error', '208/206 corrected BB error%',
        '206/204 corrected', '206/204 corrected BB error', '206/204 corrected BB error%',
        '207/204 corrected', '207/204 corrected BB error', '207/204 corrected BB error%',
       '208/204 corrected',  '208/204 corrected BB error', '208/204 corrected BB error%',
       # '206/238 Age [Ma]','207/206 Age [Ma]',
                   ]
        
        
    result_plotter = ['206/238 corrected', '206/238 corrected BB error', '206/238 corrected BB error%',
       '208/232 corrected', '208/232 corrected BB error','208/232 corrected BB error%',
        '207/206 corrected', '207/206 corrected BB error', '207/206 corrected BB error%',
       '208/206 corrected', '208/206 corrected BB error', '208/206 corrected BB error%',
        '206/204 corrected', '206/204 corrected BB error', '206/204 corrected BB error%',
        '207/204 corrected', '207/204 corrected BB error', '207/204 corrected BB error%',
       '208/204 corrected',  '208/204 corrected BB error', '208/204 corrected BB error%']
           
    grouper_plotter = ['206/238 corrected_mean','206/238 corrected_2σ',
       '206/238 corrected BB error_mean',
       '208/232 corrected_mean','208/232 corrected_2σ','208/232 corrected BB error_mean', 
       '207/206 corrected_mean', '207/206 corrected_2σ', '207/206 corrected BB error_mean',
        '208/206 corrected_mean', '208/206 corrected_2σ','208/206 corrected BB error_mean', 
        '206/204 corrected_mean','206/204 corrected_2σ', '206/204 corrected BB error_mean', 
      '207/204 corrected_mean','207/204 corrected_2σ', '207/204 corrected BB error_mean', 
      '208/204 corrected_mean', '208/204 corrected_2σ', '208/204 corrected BB error_mean', 
        ]
    
    result = result[result_order]
    grouper_comb_final = grouper_comb_final[grouper_order]
                                                        
    result_plot = result[result_plotter]
    grouper_plot = grouper_comb_final[grouper_plotter]                                                    
                                                                                                           
    
    return result, grouper_comb_final, result_plot, grouper_plot

def AB_error2(result_df):
    ''' Progress toward calculating the Constant Ext Error from Isoplot?'''
    pd.set_option('mode.chained_assignment',None)
    
    AB_err_tester = result_df
    result_headers = [
        '206/238 corrected',
        '208/232 corrected',
        '207/206 corrected',
        '208/206 corrected',
        '206/204 corrected',
        '207/204 corrected',
        '208/204 corrected'    
    ]

    for ratio in result_headers:
        #print(ratio)
        error_string = str(ratio.split()[0]) + '_after rejection 2σ%'
        new_string1 = ratio + ' 2σ'
        #print(error_string)
        new_string2 = ratio + ' x/(σ^2)'
        new_string3 = ratio + ' 1/(σ^2)'

        AB_err_tester[new_string1] = AB_err_tester.apply(lambda x: (x[error_string] / 100 ) * (x[ratio]), axis=1)

        AB_err_tester[new_string2] = AB_err_tester.apply(lambda x: x[ratio] / ((x[new_string1] / 2)**2), axis=1)
        AB_err_tester[new_string3] = AB_err_tester.apply(lambda x: (x[new_string2] / x[ratio]), axis=1)


    samples = AB_err_tester.index.values.tolist()
    group = None
    group_names = []
    start = [0]
    end = []
    for idx in range(len(samples)):
        if group == None:
            group = samples[idx].split(' 1')[0]
            group_names.append(group)
        #print(samples[idx])
        if group in samples[idx]:
            pass
        else:
            end.append(idx)
            start.append(idx) # + 1 was original
            name = samples[idx].split(' 1')[0]
            group = name
            group_names.append(name)
    end.append(len(samples))

    # print('start:', start)
    # print('end:', end)
    # print('group names:', group_names)

    stats_dict = {}
    corrected_stats_dict = {}
    group_df_list = []
    
    for idx in range(len(group_names)):
        group_dict = {}
        #print(idx)
        group_df = AB_err_tester.iloc[start[idx]:end[idx]]
        #print(group_df.index.values.tolist())
        for ratio in result_headers:
            new_string2 = ratio + ' x/(σ^2)'
            new_string3 = ratio + ' 1/(σ^2)'
            new_string4 = ratio + ' Weighted Mean'
            new_string5 = ratio + ' σ^2'
            new_string6 = ratio + ' σ'
            wtd_mean = group_df[new_string2].sum() / group_df[new_string3].sum()
            sigma_sqrd = 1 / group_df[new_string3].sum()
            sigma = sigma_sqrd ** 0.5

            group_dict[new_string4] = wtd_mean
            group_dict[new_string5] = sigma_sqrd
            group_dict[new_string6] = sigma
            
            z_string = str(ratio.split()[0]) + '_Z'
            group_df[z_string] = group_df.apply(lambda x: (((x[ratio] - group_dict[new_string4])**2) * x[new_string3] ), axis=1)
            
            new_string5b = ratio +' S-factor'
            #new_string6 = ratio + ' count'
            new_string7b = ratio + ' MSWD'
            #new_string8 = ratio + ' MSWD_difference_from_1'

            S_factor = group_df[z_string].sum()
            MSWD = S_factor / (group_df[ratio].count() - 1)
            #MSWD_dif = 1 - MSWD

            group_dict[new_string5b] = S_factor
            group_dict[new_string7b] = MSWD
        
        group_df_list.append(group_df)

        stats_dict[group_names[idx]] = group_dict

    grouper = pd.DataFrame(stats_dict) ### Will need to flip this at some point.
    new_AB_err_tester = pd.concat(group_df_list)
    
    grouper_flip = pd.DataFrame.transpose(grouper)

    return new_AB_err_tester, grouper_flip 


def stat_rank_and_correct(calc_dict, rank_perc = 20, std = 'glass'):
    
    stat_tester = statistics_ranktest2(calc_dict, rank_perc)
    result, grouper_comb_final, result_plot, grouper_plot = stat_correcter2(stat_tester, std)
    
    #print('Debug')
    stat1_order = ['238_CPS_mean', '238_CPS_2σ%', '232_CPS_mean', '232_CPS_2σ%',
       '208_CPS_mean', '208_CPS_2σ%', '207_CPS_mean', '207_CPS_2σ%',
       '206_CPS_mean', '206_CPS_2σ%', '204_CPS_mean', '204_CPS_2σ%',
       '202_CPS_mean', '202_CPS_2σ%', 'OPZ_238_mean', 'OPZ_232_mean',
       'OPZ_208_mean', 'OPZ_207_mean', 'OPZ_206_mean', 'OPZ_204_mean',
       'OPZ_202_mean','OPZ_208/206_mean','OPZ_207/206_mean','OPZ_206/204_Hg-corrected_mean',
        'SNR_238_mean', 'SNR_232_mean',
       'SNR_208_mean', 'SNR_207_mean', 'SNR_206_mean', 'SNR_204_mean',
       'SNR_202_mean', '206/238_before rejection',
       '206/238_before rejection 2se%',  '208/232_before rejection',
       '208/232_before rejection 2se%',  '207/206_before rejection',
       '207/206_before rejection 2se%',  '208/206_before rejection',
       '208/206_before rejection 2se%',  '206/204_before rejection',
       '206/204_before rejection 2se%',  '208/204_before rejection',
       '208/204_before rejection 2se%',  '207/204_before rejection',
       '207/204_before rejection 2se%',  'Time (s)']
    stat1 = stat_tester[stat1_order]
    
    stat2 = pd.concat([stat1, result], axis=1)
    
    stat_new, group2 = AB_error2(stat2)
    
    stat2_order = ['Time (s)','238_CPS_mean', '238_CPS_2σ%', '232_CPS_mean', '232_CPS_2σ%',
    '208_CPS_mean', '208_CPS_2σ%', '207_CPS_mean', '207_CPS_2σ%',
       '206_CPS_mean', '206_CPS_2σ%', '204_CPS_mean', '204_CPS_2σ%',
       '202_CPS_mean', '202_CPS_2σ%', 'OPZ_238_mean', 'OPZ_232_mean',
       'OPZ_208_mean', 'OPZ_207_mean', 'OPZ_206_mean', 'OPZ_204_mean',
       'OPZ_202_mean', 'OPZ_208/206_mean','OPZ_207/206_mean','OPZ_206/204_Hg-corrected_mean',
        'SNR_238_mean', 'SNR_232_mean',
       'SNR_208_mean', 'SNR_207_mean', 'SNR_206_mean', 'SNR_204_mean',
       'SNR_202_mean', '206/238_before rejection',
       '206/238_before rejection 2se%',  '208/232_before rejection',
       '208/232_before rejection 2se%',  '207/206_before rejection',
       '207/206_before rejection 2se%',  '208/206_before rejection',
       '208/206_before rejection 2se%',  '206/204_before rejection',
       '206/204_before rejection 2se%',  '208/204_before rejection',
       '208/204_before rejection 2se%',  '207/204_before rejection', '207/204_before rejection 2se%',  
       '206/238_after rejection', '206/238_after rejection 2se%','206/238_after rejection 2se',
       '208/232_after rejection', '208/232_after rejection 2se%','208/232_after rejection 2se',
       '207/206_after rejection', '207/206_after rejection 2se%','207/206_after rejection 2se',
       '208/206_after rejection', '208/206_after rejection 2se%','208/206_after rejection 2se',
       '206/204_after rejection', '206/204_after rejection 2se%','206/204_after rejection 2se',
       '208/204_after rejection', '208/204_after rejection 2se%','208/204_after rejection 2se',
       '207/204_after rejection', '207/204_after rejection 2se%','207/204_after rejection 2se',
       '206/238 corrected', '206/238 corrected 2σ','206/238 corrected x/(σ^2)', '206/238 corrected 1/(σ^2)', '206/238 corrected BB error', '206/238 corrected BB error%',
       '208/232 corrected', '208/232 corrected 2σ', '208/232 corrected x/(σ^2)','208/232 corrected 1/(σ^2)', '208/232 corrected BB error','208/232 corrected BB error%',
       '207/206 corrected', '207/206 corrected 2σ','207/206 corrected x/(σ^2)', '207/206 corrected 1/(σ^2)','207/206 corrected BB error', '207/206 corrected BB error%',
       '208/206 corrected', '208/206 corrected 2σ', '208/206 corrected x/(σ^2)','208/206 corrected 1/(σ^2)', '208/206 corrected BB error', '208/206 corrected BB error%',
       '206/204 corrected', '206/204 corrected 2σ', '206/204 corrected x/(σ^2)', '206/204 corrected 1/(σ^2)', '206/204 corrected BB error', '206/204 corrected BB error%',
       '207/204 corrected','207/204 corrected 2σ', '207/204 corrected x/(σ^2)', '207/204 corrected 1/(σ^2)', '207/204 corrected BB error', '207/204 corrected BB error%',
       '208/204 corrected', '208/204 corrected 2σ','208/204 corrected x/(σ^2)', '208/204 corrected 1/(σ^2)',  '208/204 corrected BB error', '208/204 corrected BB error%',          
       '206/238_Z','208/232_Z', '207/206_Z', '208/206_Z', '206/204_Z', '207/204_Z','208/204_Z', 
                  # '206/238 Age [Ma]','207/206 Age [Ma]',
        ]
    
    new_grouper_plot2 = group2[[
        '206/238 corrected Weighted Mean','206/238 corrected MSWD',
        '208/232 corrected Weighted Mean','208/232 corrected MSWD',
        '207/206 corrected Weighted Mean','207/206 corrected MSWD',
        '208/206 corrected Weighted Mean','208/206 corrected MSWD',
        '206/204 corrected Weighted Mean','206/204 corrected MSWD',
        '207/204 corrected Weighted Mean','207/204 corrected MSWD',
        '208/204 corrected Weighted Mean','208/204 corrected MSWD',   
    ]]
    
    new_grouper_plot = pd.concat([grouper_plot,new_grouper_plot2], axis = 1)
    #print('DEBUG')
    
    stat2 = stat_new[stat2_order]
    
    group_new = pd.concat([grouper_comb_final, group2], axis = 1)
    
    return stat2, group_new, result_plot, new_grouper_plot


def files_ranked_toEXCEL(calc_dict, excel_name):
    stats = statistics_ranktest(calc_dict)
    with pd.ExcelWriter(excel_name) as writer:
        for sheet in calc_dict:
            calc_dict[sheet].to_excel(writer, sheet_name = sheet, index = False)
        
        stats.to_excel(writer, sheet_name = 'Statistics', index = True)
    new_filename = str(excel_name.split('.')[0]) + '_statistics.xlsx'
    with pd.ExcelWriter(new_filename) as writer:
        stats.to_excel(writer, sheet_name = 'Statistics', index = True)

def files_process_toEXCEL(calc_dict, excel_name):
    with pd.ExcelWriter(excel_name) as writer:
        for sheet in calc_dict:
            calc_dict[sheet].to_excel(writer, sheet_name = sheet, index = False)
        
        statistics_NP2(calc_dict).to_excel(writer, sheet_name = 'Statistics', index = True)

def file_process_combine(filename):
    calc_dict = calc_CPS(read_np2_timeseries(filename))
    new_filename = str(filename.split('.')[0]) + '_processed.xlsx'
    files_process_toEXCEL(calc_dict, new_filename)
    

In [5]:
#Functons for graphing and report generation
def U_Pb_plots(calc_dict, sample, choice = True):
    key_list = ['238_CPS', '232_CPS',
       '208_CPS', '207_CPS', '206_CPS', '204_CPS', '202_CPS', '206/238',
       '208/232', '207/206', '208/206', '206/204']
    
    zet = calc_dict[sample]
    new_string = sample.split('time')[0].rstrip()
    y_list = []
    for key in key_list:
        y_list.append(zet[key])
    
    x = zet['Elapsed Time']
    
    fig, axs = plt.subplots(4, 3, sharex = True, figsize = (12, 12))
    fig.suptitle(new_string, fontsize=24)
    
    ax_list = [
        axs[0, 0], 
        axs[0, 1],   
        axs[0, 2], 
        axs[1, 0], 
        axs[1, 1],
        axs[1, 2],
        axs[2, 0], 
        axs[2, 1], 
        axs[2, 2],
        axs[3, 0], 
        axs[3, 1], 
        axs[3, 2]   
        ]

    axs[0, 0].plot(x, y_list[0])
    axs[0, 1].plot(x, y_list[1])
    axs[0, 2].plot(x, y_list[2])
    axs[1, 0].plot(x, y_list[3])
    axs[1, 1].plot(x, y_list[4])
    axs[1, 2].plot(x, y_list[5])
    axs[2, 0].plot(x, y_list[6])
    axs[2, 1].plot(x, y_list[7])
    axs[2, 2].plot(x, y_list[8])
    axs[3, 0].plot(x, y_list[9])
    axs[3, 0].set(xlabel = 'Time (s)')
    axs[3, 1].plot(x, y_list[10])
    axs[3, 1].set(xlabel = 'Time (s)')
    axs[3, 2].plot(x, y_list[11])
    axs[3, 2].set(xlabel = 'Time (s)')
    for idx in range(len(ax_list)):
        ax_list[idx].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax_list[idx].set_title(key_list[idx])
        y_mean = [np.mean(y_list[idx])]*len(x)
        # Plot the average line
        mean_line = ax_list[idx].plot(x,y_mean, label='Mean', linestyle='--', color = "black")
        # Make a legend
        legend = ax_list[idx].legend(loc='upper right')
    
    MYDIR = ("Figures")
    CHECK_FOLDER = os.path.isdir(MYDIR)

    # If folder doesn't exist, then create it.
    if not CHECK_FOLDER:
        os.makedirs(MYDIR)
        #print("created folder : ", MYDIR)
    
    #new_string = sample.replace('time series data', '').rstrip()
    
    filename = os.path.join(MYDIR, new_string + '.pdf')
    plt.savefig(filename)
    print('Plot for ', new_string, " is complete.")
    
    if choice == False:
        plt.close()
    #else:
        #plt.close()
        

def U_Pb_report(calc_dict, intro_filename, intro = False, output_name = 'U-Pb_output.pdf'):
    MYDIR = ("Figures")
    mergedObject = PdfFileMerger()
    
    if intro:
        mergedObject.append(PdfFileReader(intro_filename, 'rb'))
        print(f'Succesfully incorporated {intro_filename} into PDF.')
    pd.set_option('precision', 2)
    stats = statistics_NP2(calc_dict)
    stat_dict = {}
    stat_dict['stat1'] = stats.iloc[:, 14:]
    stat_dict['stat2'] = stats.iloc[:, :8]
    stat_dict['stat3'] = stats.iloc[:, 8:14]
    html_list = []

    for key in stat_dict:
        name = key + ".pdf"
        stats_html = stat_dict[key].to_html()
        pdfkit.from_string(stats_html, name)
        mergedObject.append(PdfFileReader(name, 'rb'))

    file_list = []
    keys = calc_dict.keys()

    for key in keys:
        #print(key)
        U_Pb_plots(calc_dict, key, False)
        new_string = key.split('time')[0].rstrip()
        filename = os.path.join(MYDIR, new_string + '.pdf')

        mergedObject.append(PdfFileReader(filename, 'rb'))

    if '.pdf' in output_name:
        pass
    else:
        output_name = output_name + '.pdf'
    
    #output_name = "U-Pb_output.pdf"  
    mergedObject.write(output_name)

    print(f'PDF file named: {output_name} is complete.')        
        


In [6]:
#Published values dictionary

published_ratios_610 = {
    '206/238': 0.258174418887103,
    '208/232': 0.546910763260703,
    '207/206': 0.909778846717897,
    '208/206': 2.16900334369684,
    '206/204': 17.047,
    '207/204': 15.509,
    '208/204': 36.975,

}
published_ratios_612 = {
    '206/238': 0.289090155580635,
    '208/232': 0.598868250924868,
    '207/206': 0.907335907335907,
    '208/206': 2.16450216450216,
    '206/204': 17.094,
    '207/204': 15.51,
    '208/204': 37,      
}
published_ratios_BHVO = {
    '206/238': 1.24298582359808,
    '208/232': 0.811596334965975,
    '207/206': 0.831206960977953,
    '208/206': 2.042918913147926,
    '206/204': 18.733,
    '207/204': 15.571,
    '208/204': 38.27,      
}   
published_ratios_BCR_2G = {
    '206/238': 1.90697056349179,
    '208/232': 1.09082521297421,
    '207/206': 0.832720490274447,
    '208/206': 2.06394884092726,
    '206/204': 18.765,
    '207/204': 15.626,
    '208/204': 38.73,      
}
published_ratios_GSD_1G = {
    '206/238': 0.367307298285922,
    '208/232': 0.706245210573853,
    '207/206': 0.80417794575821,
    '208/206': 1.98723121712038,
    '206/204': 19.579,
    '207/204': 15.745,
    '208/204': 38.908,      
}
published_ratios_GSE_1G = { ###These are just the GSD-1G ratios!!!!!
    '206/238': 0.367307298285922,
    '208/232': 0.706245210573853,
    '207/206': 0.80417794575821,
    '208/206': 1.98723121712038,
    '206/204': 19.579,
    '207/204': 15.745,
    '208/204': 38.908,      
}
published_ratios_MKED = {
    '206/238': 0.26538,
    '208/232': 0.08127,
    '207/206': 0.09465,
    '208/206': 0.8719,
    '206/204': 18962.2,
    '207/204': 1794.77223,
    '208/204': 16533.14218,      
}
published_ratios_BLR_1 = {
    '206/238': 0.176364,
    '208/232': 0.0533134203278787,
    '207/206': 0.0743078,
    '208/206': 0.17982,
    '206/204': 975.36,
    '207/204': 72.47686,
    '208/204': 175.38924,      
}
published_ratios_OLT_1 = {
    '206/238': 0.170681666666667,
    '208/232': 0.0516589274888366,
    '207/206': 0.0731801314551979,
    '208/206': 0.58025,
    '206/204': 1847.16667,
    '207/204': 135.17590,
    '208/204': 1071.81688,      
}
published_ratios_OLT_2 = {
    '206/238': 0.16693,
    '208/232': 0.05064,
    '207/206': 0.07248,
    '208/206': 0.63815,
    '206/204': 1453.00000,
    '207/204': 105.31206,
    '208/204': 927.22713,      
}
published_ratios_TCB = {
    '206/238': 0.17086,
    '208/232': 0.05178,
    '207/206': 0.07327,
    '208/206': 0.63348,
    '206/204': 2016.75000,
    '207/204': 147.76034,
    '208/204': 1277.56162,      
}
published_ratios_MudTank = {
    '206/238': 0.052269449,
    '208/232': 0.049381079,
    '207/206': 0.071923689,
    '208/206': 0.068164043,
    '206/204': 766.0145618,
    '207/204': 55.09459,
    '208/204': 52.21465,      
}

published_ratios_dict = {
    'SRM NIST 610': published_ratios_610,
    'SRM NIST 612': published_ratios_612,
    'BHVO-2G': published_ratios_BHVO,
    'BCR-2G': published_ratios_BCR_2G,
    'GSD-1G': published_ratios_GSD_1G,
    'GSE-1G': published_ratios_GSE_1G,
    'Bear Lake': published_ratios_BLR_1,
    'MKED': published_ratios_MKED,
    'Mudtank': published_ratios_MudTank,
    'OLT-1': published_ratios_OLT_1,
    'OLT-2': published_ratios_OLT_2,
    'TCB': published_ratios_TCB,
}

published_df = pd.DataFrame(published_ratios_dict)
published_df = pd.DataFrame.transpose(published_df)

#Developing chronologic function

#Check order in Iolite

chrono_order = ['SRM NIST 610 1.1',
 'SRM NIST 610 1.2',
 'SRM NIST 610 1.3',
 'SRM NIST 612 1.1',
 'SRM NIST 612 1.2',
 'SRM NIST 612 1.3',
 'BHVO-2G 1.1',
 'BHVO-2G 1.2',
 'BHVO-2G 1.3',  
 'BCR-2G 1.1',
 'BCR-2G 1.2',
 'BCR-2G 1.3',                          
 'Bear Lake 1.1',
 'Bear Lake 1.2',
 'Bear Lake 1.3',
 'MKED 1.1',
 'MKED 1.2',
 'MKED 1.3',              
 'Bancroft 1.1',
 'Bancroft 1.2',
 'Bancroft 1.3',
 'GSD-1G 1.1',
 'GSD-1G 1.2',
 'GSD-1G 1.3',              
 'GSE-1G 1.1',
 'GSE-1G 1.2',
 'GSE-1G 1.3',              
 
                
 'SRM NIST 610 1.4',
 'SRM NIST 610 1.5',
 'SRM NIST 610 1.6', 
 'SRM NIST 612 1.4',
 'SRM NIST 612 1.5',
 'SRM NIST 612 1.6',              
 'BHVO-2G 1.4',
 'BHVO-2G 1.5',
 'BHVO-2G 1.6', 
 'BCR-2G 1.4',
 'BCR-2G 1.5',
 'BCR-2G 1.6',                  
 'Bear Lake 1.4',
 'Bear Lake 1.5',
 'Bear Lake 1.6',              
 'MKED 1.4',
 'MKED 1.5',
 'MKED 1.6', 
 'Bancroft 1.4',
 'Bancroft 1.5',
 'Bancroft 1.6', 
 'Mudtank 1.1',
 'Mudtank 1.2',
 'Mudtank 1.3',  
 'Yates 1.1',
 'Yates 1.2',
 'Yates 1.3', 
                
 'Bancroft 1.7',
 'Bancroft 1.8',
 'Bancroft 1.9',
 'MKED 1.7',
 'MKED 1.8',
 'MKED 1.9',
 'Bear Lake 1.7',
 'Bear Lake 1.8',
 'Bear Lake 1.9',              
 'BCR-2G 1.7',
 'BCR-2G 1.8',
 'BCR-2G 1.9',
 'BHVO-2G 1.7',
 'BHVO-2G 1.8',
 'BHVO-2G 1.9',              
 'SRM NIST 612 1.7',
 'SRM NIST 612 1.8',
 'SRM NIST 612 1.9',                              
 'SRM NIST 610 1.7',
 'SRM NIST 610 1.8',
 'SRM NIST 610 1.9',
 
 ]


def chronologic(result_df, order = chrono_order):
    new_result = result_df.copy(deep=True)
    
    return new_result.reindex(order)

In [7]:
filename = '06May_ttnSS2_NP2_baseline_corrected.xlsx'
test_df = read_np2_timeseries(filename)

In [8]:
tester = calc_CPS(test_df)
tester.keys()

dict_keys(['SRM NIST 610 1.1', 'SRM NIST 610 1.2', 'SRM NIST 610 1.3', 'SRM NIST 610 1.4', 'SRM NIST 610 1.5', 'SRM NIST 610 1.6', 'SRM NIST 610 1.7', 'SRM NIST 610 1.8', 'SRM NIST 610 1.9', 'SRM NIST 612 1.1', 'SRM NIST 612 1.2', 'SRM NIST 612 1.3', 'SRM NIST 612 1.4', 'SRM NIST 612 1.5', 'SRM NIST 612 1.6', 'SRM NIST 612 1.7', 'SRM NIST 612 1.8', 'SRM NIST 612 1.9', 'BHVO-2G 1.1', 'BHVO-2G 1.2', 'BHVO-2G 1.3', 'BHVO-2G 1.4', 'BHVO-2G 1.5', 'BHVO-2G 1.6', 'BHVO-2G 1.7', 'BHVO-2G 1.8', 'BHVO-2G 1.9', 'BCR-2G 1.1', 'BCR-2G 1.2', 'BCR-2G 1.3', 'BCR-2G 1.4', 'BCR-2G 1.5', 'BCR-2G 1.6', 'BCR-2G 1.7', 'BCR-2G 1.8', 'BCR-2G 1.9', 'GSD-1G 1.1', 'GSD-1G 1.2', 'GSD-1G 1.3', 'GSE-1G 1.1', 'GSE-1G 1.2', 'GSE-1G 1.3', 'Bear Lake 1.1', 'Bear Lake 1.2', 'Bear Lake 1.3', 'Bear Lake 1.4', 'Bear Lake 1.5', 'Bear Lake 1.6', 'Bear Lake 1.7', 'Bear Lake 1.8', 'Bear Lake 1.9', 'Bancroft 1.1', 'Bancroft 1.2', 'Bancroft 1.3', 'Bancroft 1.4', 'Bancroft 1.5', 'Bancroft 1.6', 'Bancroft 1.7', 'Bancroft 1.8', 'Ba

In [ ]:
#stat_tester.keys()

In [ ]:
# tester1_1, grouper_comb_final, result_plot, grouper_plot = stat_rank_and_correct(tester, std = 'glass')
# index_list = tester1_1.keys()
# for idx in index_list:
#     print(idx)

In [ ]:
glass612_zero_full, glass612_zero_groups, glass612_zero_full_plot, glass612_zero_groups_plot  = stat_rank_and_correct(tester, 0, '612')

In [ ]:
glass612_zero_groups.keys()

In [ ]:
test_guy = glass612_zero_groups[['206/238_mean', '207/206_mean']]

keys = test_guy.index.values.tolist()
new_keys = ['Bear Lake', 'Bancroft', 'Yates', 'Mudtank', 'MKED']
# for key in new_keys:
    
#     input2 = test_guy.loc[key]['206/238_mean']
#     age = age206vs238_calc(input2, 1000*(10**6), 6) /1000000
#     print(key, ' ',age)

input2 = test_guy.loc['MKED']['206/238_mean']
age206vs238_calc(input2, 1200E6, 6) /1000000

In [9]:
### Takes  2:20 to run... be careful
glass_zero_full, glass_zero_groups, glass_zero_full_plot, glass_zero_groups_plot  = stat_rank_and_correct(tester, 0, 'glass')
glass_20_full, glass_20_groups, glass_20_full_plot, glass_20_groups_plot = stat_rank_and_correct(tester, 20, 'glass')
glass_40_full, glass_40_groups, glass_40_full_plot, glass_40_groups_plot = stat_rank_and_correct(tester, 40, 'glass')

glass612_zero_full, glass612_zero_groups, glass612_zero_full_plot, glass612_zero_groups_plot  = stat_rank_and_correct(tester, 0, '612')
glass612_20_full, glass612_20_groups, glass612_20_full_plot, glass612_20_groups_plot = stat_rank_and_correct(tester, 20, '612')
glass612_40_full, glass612_40_groups, glass612_40_full_plot, glass612_40_groups_plot = stat_rank_and_correct(tester, 40, '612')

glassBHVO_zero_full, glassBHVO_zero_groups, glassBHVO_zero_full_plot, glassBHVO_zero_groups_plot  = stat_rank_and_correct(tester, 0, 'BHVO')
glassBHVO_20_full, glassBHVO_20_groups, glassBHVO_20_full_plot, glassBHVO_20_groups_plot = stat_rank_and_correct(tester, 20, 'BHVO')
glassBHVO_40_full, glassBHVO_40_groups, glassBHVO_40_full_plot, glassBHVO_40_groups_plot = stat_rank_and_correct(tester, 40, 'BHVO')


ttn_zero_full, ttn_zero_groups, ttn_zero_full_plot, ttn_zero_groups_plot  = stat_rank_and_correct(tester, 0, 'ttn')
ttn_20_full, ttn_20_groups, ttn_20_full_plot, ttn_20_groups_plot = stat_rank_and_correct(tester, 20, 'ttn')
ttn_40_full, ttn_40_groups, ttn_40_full_plot, ttn_40_groups_plot = stat_rank_and_correct(tester, 40, 'ttn')





610 primary std
610 primary std
610 primary std
612 primary std
612 primary std
612 primary std
BHVO primary std
BHVO primary std
BHVO primary std
MKED primary std
MKED primary std
MKED primary std


In [16]:
keys = glassBHVO_20_groups.keys()
for key in keys:
    print(key)

206/238_mean
206/238_2σ%
206/238_2se%
208/232_mean
208/232_2σ%
208/232_2se%
207/206_mean
207/206_2σ%
207/206_2se%
208/206_mean
208/206_2σ%
208/206_2se%
206/204_mean
206/204_2σ%
206/204_2se%
208/204_mean
208/204_2σ%
208/204_2se%
207/204_mean
207/204_2σ%
207/204_2se%
206/238 corrected_mean
206/238 corrected_2σ
206/238 corrected_2se%
206/238 corrected_2se
206/238 corrected_2σ%
206/238 corrected BB error_mean
206/238 corrected BB error%_mean
208/232 corrected_mean
208/232 corrected_2σ
208/232 corrected_2σ%
208/232 corrected_2se%
208/232 corrected_2se
208/232 corrected BB error_mean
208/232 corrected BB error%_mean
207/206 corrected_mean
207/206 corrected_2σ
207/206 corrected_2σ%
207/206 corrected_2se%
207/206 corrected_2se
207/206 corrected BB error_mean
207/206 corrected BB error%_mean
208/206 corrected_mean
208/206 corrected_2σ
208/206 corrected_2σ%
208/206 corrected_2se%
208/206 corrected_2se
208/206 corrected BB error_mean
208/206 corrected BB error%_mean
206/204 corrected_mean
206/204

In [17]:
list_ratios = ['206/238 corrected_mean',
       '208/232 corrected_mean', '207/206 corrected_mean',  '208/206 corrected_mean', 
        '206/204 corrected_mean', '207/204 corrected_mean','208/204 corrected_mean']
    


current_dict = { 'glass_zero_groups':glass_zero_groups[list_ratios],
               
               'glass_20_groups': glass_20_groups[list_ratios],
                
               'glass_40_groups':glass_40_groups[list_ratios],
               
              
               'glass612_zero_groups':glass612_zero_groups[list_ratios],
                
               'glass612_20_groups': glass612_20_groups[list_ratios],
               
               'glass612_40_groups':glass612_40_groups[list_ratios],
               
                
               'glassBHVO_zero_groups':glassBHVO_zero_groups[list_ratios],
               
               'glassBHVO_20_groups': glassBHVO_20_groups[list_ratios],
                
               'glassBHVO_40_groups':glassBHVO_40_groups[list_ratios]
    
    
    
}

In [18]:
excel_name = 'Processed_LASS_data_difRejects_difStds_dif207206.xlsx'
with pd.ExcelWriter(excel_name, engine = 'xlsxwriter') as writer:
      # Get the xlsxwriter workbook objects.
    workbook  = writer.book
    col_format1 = workbook.add_format()
    col_format2 = workbook.add_format()
    col_format3 = workbook.add_format()
    col_format1.set_bg_color('#E6E6FA') #Lavender
    col_format2.set_bg_color('#98FB98') #Pale Green
    col_format3.set_bg_color('#98FB98') #Antique White
    
    
    for sheet in current_dict:
        current_dict[sheet].to_excel(writer, sheet_name = sheet, index = True)
        worksheet = writer.sheets[sheet]  # pull worksheet object
        test_list = list(current_dict[sheet].keys())
        worksheet.set_column(0, 0, 20)
        worksheet.freeze_panes(1,1)
        for idx in enumerate(test_list):
            if 'full' in sheet:
                if 'BB' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)
                elif 'corrected' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format1)
                elif 'SNR_20' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format2)
                    
                else:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)
            elif 'groups' in sheet:
                if 'BB' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)
                elif 'mean' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format2)
                elif 'MSWD' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format3)
                elif 'Weighted' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format1)        
                else:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)

In [ ]:
export_dict = {'glass_zero_full':glass_zero_full, 
                'glass_zero_groups':glass_zero_groups,
               'glass_20_full': glass_20_full, 
               'glass_20_groups': glass_20_groups,
               'glass_40_full':glass_40_full, 
               'glass_40_groups':glass_40_groups,
               
               'glass612_zero_full':glass612_zero_full, 
               'glass612_zero_groups':glass612_zero_groups,
               'glass612_20_full': glass612_20_full, 
               'glass612_20_groups': glass612_20_groups,
               'glass612_40_full':glass612_40_full, 
               'glass612_40_groups':glass612_40_groups,
               
               'glassBHVO_zero_full':glassBHVO_zero_full, 
               'glassBHVO_zero_groups':glassBHVO_zero_groups,
               'glassBHVO_20_full': glassBHVO_20_full, 
               'glassBHVO_20_groups': glassBHVO_20_groups,
               'glassBHVO_40_full':glassBHVO_40_full, 
               'glassBHVO_40_groups':glassBHVO_40_groups,
               
               'ttn_zero_full': ttn_zero_full, 
               'ttn_zero_groups': ttn_zero_groups,
               'ttn_20_full': ttn_20_full, 
               'ttn_20_groups': ttn_20_groups,
               'ttn_40_full': ttn_40_full, 
               'ttn_40_groups':ttn_40_groups
              }

plot_dict = {'glass_zero_full':glass_zero_full_plot, 
                'glass_zero_groups':glass_zero_groups_plot,
               'glass_20_full': glass_20_full_plot, 
               'glass_20_groups': glass_20_groups_plot,
               'glass_40_full':glass_40_full_plot, 
               'glass_40_groups':glass_40_groups_plot,
             
               'glass612_zero_full':glass612_zero_full_plot, 
               'glass612_zero_groups':glass612_zero_groups_plot,
               'glass612_20_full': glass612_20_full_plot, 
               'glass612_20_groups': glass612_20_groups_plot,
               'glass612_40_full':glass612_40_full_plot, 
               'glass612_40_groups':glass612_40_groups_plot,
             
               'glassBHVO_zero_full':glassBHVO_zero_full_plot, 
               'glassBHVO_zero_groups':glassBHVO_zero_groups_plot,
               'glassBHVO_20_full': glassBHVO_20_full_plot, 
               'glassBHVO_20_groups': glassBHVO_20_groups_plot,
               'glassBHVO_40_full':glassBHVO_40_full_plot, 
               'glassBHVO_40_groups':glassBHVO_40_groups_plot,
             
               'ttn_zero_full': ttn_zero_full_plot, 
               'ttn_zero_groups': ttn_zero_groups_plot,
               'ttn_20_full': ttn_20_full_plot, 
               'ttn_20_groups': ttn_20_groups_plot,
               'ttn_40_full': ttn_40_full_plot, 
               'ttn_40_groups':ttn_40_groups_plot
              }

chrono_dict = {
            'glass_zero_full': chronologic(glass_zero_full), 
            'glass_zero_groups': glass_zero_groups,
                }

In [ ]:
###Export for full data worksheets


excel_name = 'Processed_LASS_data_difRejects_difStds_newTest3.xlsx'
with pd.ExcelWriter(excel_name, engine = 'xlsxwriter') as writer:
      # Get the xlsxwriter workbook objects.
    workbook  = writer.book
    col_format1 = workbook.add_format()
    col_format2 = workbook.add_format()
    col_format3 = workbook.add_format()
    col_format1.set_bg_color('#E6E6FA') #Lavender
    col_format2.set_bg_color('#98FB98') #Pale Green
    col_format3.set_bg_color('#98FB98') #Antique White
    
    
    for sheet in export_dict:
        export_dict[sheet].to_excel(writer, sheet_name = sheet, index = True)
        worksheet = writer.sheets[sheet]  # pull worksheet object
        test_list = list(export_dict[sheet].keys())
        worksheet.set_column(0, 0, 20)
        worksheet.freeze_panes(1,1)
        for idx in enumerate(test_list):
            if 'full' in sheet:
                if 'BB' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)
                elif 'corrected' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format1)
                elif 'SNR_20' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format2)
                    
                else:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)
            elif 'groups' in sheet:
                if 'BB' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)
                elif 'mean' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format2)
                elif 'MSWD' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format3)
                elif 'Weighted' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format1)        
                else:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)
                    


In [ ]:
###Export for plotting worksheets


excel_name = 'Processed_LASS_data_difRejects_difStds_plottingNEW2.xlsx'

with pd.ExcelWriter(excel_name, engine = 'xlsxwriter') as writer:
    # Get the xlsxwriter workbook objects.
    workbook  = writer.book
    col_format1 = workbook.add_format()
    col_format2 = workbook.add_format()
    col_format3 = workbook.add_format()
    col_format1.set_bg_color('#E6E6FA') #Lavender
    col_format2.set_bg_color('#98FB98') #Pale Green
    col_format3.set_bg_color('#98FB98') #Antique White
    
    for sheet in plot_dict:
        
        plot_dict[sheet].to_excel(writer, sheet_name = sheet, index = True)
        worksheet = writer.sheets[sheet]  # pull worksheet object
        test_list = list(plot_dict[sheet].keys())
        worksheet.set_column(0, 0, 20)
        worksheet.freeze_panes(1,1)
        for idx in enumerate(test_list):
            if 'full' in sheet:
                if 'BB' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)
                else:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format1)
            elif 'groups' in sheet:
                if 'BB' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)
                elif 'mean' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format1)
                elif 'MSWD' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format3)
                elif 'Weighted' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format2)         
                else:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)
    

In [ ]:
###Export for chronologic worksheets

excel_name = 'Processed_LASS_data_difRejects_difStds_Chrono_test1.xlsx'
with pd.ExcelWriter(excel_name, engine = 'xlsxwriter') as writer:
      # Get the xlsxwriter workbook objects.
    workbook  = writer.book
    col_format1 = workbook.add_format()
    col_format2 = workbook.add_format()
    col_format1.set_bg_color('#E6E6FA') #Lavender
    col_format2.set_bg_color('#98FB98') #Pale Green
    
    for sheet in chrono_dict:

        chrono_dict[sheet].to_excel(writer, sheet_name = sheet, index = True)
        worksheet = writer.sheets[sheet]  # pull worksheet object
        test_list = list(chrono_dict[sheet].keys())
        worksheet.set_column(0, 0, 20)
        worksheet.freeze_panes(1,1)
        for idx in enumerate(test_list):
            if 'full' in sheet:
                if 'BB' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)
                elif 'corrected' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format1)
                elif 'SNR_20' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format2)
                elif 'OPZ' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format1)   
                else:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)
            elif 'groups' in sheet:
                if 'BB' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)
                elif 'mean' in idx[1]:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20, col_format2)
                else:
                    worksheet.set_column(idx[0]+1, idx[0]+1, 20)    


In [ ]:
### plotting?


#What do I input?

# plot_dict
# group_namer
#primary_std
#published_df

# def ratio_plot(group_namer, primary_std, plot_dict = plot_dict, published_df = published_df)



# Making groups of the standards for plotting
worksheets = list(plot_dict.keys())
group = None
std_names = []
start = [0]
end = []
for idx in range(len(worksheets)):
    if group == None:
        group = worksheets[idx].split('_')[0]
        std_names.append(group)
    #print(samples[idx])
    if worksheets[idx].split('_')[0] in group:
        pass
    else:
        end.append(idx)
        start.append(idx) 
        name = worksheets[idx].split('_')[0]
        group = name
        std_names.append(name)
end.append(len(worksheets))

#print('start:', start)
#print('end:', end)
#print('std names:', std_names)

std_dict = {}

for idx in range(len(std_names)):
    group_dict = {}
    std_group = worksheets[start[idx]:end[idx]]
    #print('std_group', std_group)
    group_dict['0%'] = std_group[0]
    group_dict['20%'] = std_group[2]
    group_dict['40%'] = std_group[4]
    
    group_dict['0% group'] = std_group[1]
    group_dict['20% group'] = std_group[3]
    group_dict['40% group'] = std_group[5]
    
    std_dict[std_names[idx]] = group_dict


#Creating loop for all entries in plot_dict

plot_dict_grouped_dict = {}

for key in plot_dict:
    
    if 'full' in key:
        
    #Create groups for plotting

        samples = plot_dict[key].index.values.tolist()
        group = None
        group_names = []
        start = [0]
        end = []
        for idx in range(len(samples)):
            if group == None:
                group = samples[idx].split(' 1')[0]
                group_names.append(group)
            #print(samples[idx])
            if group in samples[idx]:
                pass
            else:
                end.append(idx)
                start.append(idx) # + 1 was original
                name = samples[idx].split(' 1')[0]
                group = name
                group_names.append(name)
        end.append(len(samples))

        #print('start:', start)
        #print('end:', end)
        #print('group names:', group_names)

        group_df_list = []
        group_df_dict = {}

        for idx in range(len(group_names)):
            group_dict = {}
            #print(idx)
            group_df = plot_dict[key].iloc[start[idx]:end[idx]]
            #print(group_df.index.values.tolist())

            group_df_list.append(group_df)
            group_df_dict[group_names[idx]] = group_df
        
        plot_dict_grouped_dict[key] = group_df_dict
        
        if 'group' in key:
             plot_dict_grouped_dict[key] = plot_dict[key]


#Data to be plotted


group_namer = 'SRM NIST 612'
primary_std = 'glass612'

primary_std_name = {
    'glass': "SRM NIST 610",
    'glass612': "SRM NIST 612",
    'glassBHVO': "BHVO-2G",
    'ttn': "MKED-1",
}

test_zero = plot_dict_grouped_dict[ std_dict[primary_std]['0%']][group_namer]
test_20 = plot_dict_grouped_dict[ std_dict[primary_std]['20%']][group_namer]
test_40 = plot_dict_grouped_dict[ std_dict[primary_std]['40%']][group_namer]

test_group_zero = plot_dict[std_dict[primary_std]['0% group']].loc[group_namer]
test_group_20 = plot_dict[std_dict[primary_std]['20% group']].loc[group_namer]
test_group_40 = plot_dict[std_dict[primary_std]['40% group']].loc[group_namer]


if (group_namer == 'Bancroft') or (group_namer =='Yates'):
    #print('debug')
    pub_ratio = published_df.loc[['OLT-1', 'OLT-2', 'TCB']]
else:
    pub_ratio = published_df.loc[group_namer]


color_dict = {
    '0%': 'red',
    '20%': 'blue',
    '40%': 'green',
    'pub': 'purple'
}


#Initialize plots
fig, axs = plt.subplots(2, 2, sharex = False, figsize = (24, 12))
fig.suptitle(group_namer + ', Primary Std: ' + primary_std_name[primary_std], fontsize=28)

#207/204 vs 206/204 plot
    #Results for each line
x1_entry = '206/204 corrected'
y1_entry = '207/204 corrected'
ax = axs[0,0]
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)

ax.plot(test_zero[x1_entry], test_zero[y1_entry], 'o', color= color_dict['0%']) #0% reject
ax.plot(test_20[x1_entry], test_20[y1_entry], 'o', color= color_dict['20%']) #20% reject
ax.plot(test_40[x1_entry], test_40[y1_entry], 'o', color= color_dict['40%']) #40% reject
    #Group averages
x1_entry = '206/204 corrected_mean'
y1_entry = '207/204 corrected_mean'
ax.plot(test_group_zero[x1_entry], test_group_zero[y1_entry], '^', color= color_dict['0%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
ax.plot(test_group_20[x1_entry], test_group_20[y1_entry], '^', color= color_dict['20%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
ax.plot(test_group_40[x1_entry], test_group_40[y1_entry], '^', color= color_dict['40%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
    #Group 2sigma
x_error = '206/204 corrected_2σ'
y_error = '206/204 corrected_2σ'
ax.errorbar(test_group_zero[x1_entry], test_group_zero[y1_entry],test_group_zero[x_error], test_group_zero[y_error],color = 'black')
ax.errorbar(test_group_20[x1_entry], test_group_20[y1_entry],test_group_20[x_error], test_group_20[y_error],color = 'black')
ax.errorbar(test_group_40[x1_entry], test_group_40[y1_entry],test_group_40[x_error], test_group_40[y_error],color = 'black')
    #Published value
x1_entry = '206/204'
y1_entry = '207/204'
ax.plot(pub_ratio[x1_entry], pub_ratio[y1_entry], 's', color= color_dict['pub'], markersize=18, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

    #Formatting
ax.set(xlabel = '206/204', ylabel = '207/204',)
ax.yaxis.label.set_size(20)
ax.xaxis.label.set_size(20)


#208/204 vs 206/204 plot

x2_entry = '206/204 corrected'
y2_entry = '208/204 corrected'
ax = axs[0,1]
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)

ax.plot(test_zero[x2_entry], test_zero[y2_entry], 'o', color= color_dict['0%']) #0% reject
ax.plot(test_20[x2_entry], test_20[y2_entry], 'o', color= color_dict['20%']) #20% reject
ax.plot(test_40[x2_entry], test_40[y2_entry], 'o', color= color_dict['40%']) #40% reject

x1_entry = '206/204 corrected_mean'
y1_entry = '208/204 corrected_mean'
ax.plot(test_group_zero[x1_entry], test_group_zero[y1_entry], '^', color= color_dict['0%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
ax.plot(test_group_20[x1_entry], test_group_20[y1_entry], '^', color= color_dict['20%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
ax.plot(test_group_40[x1_entry], test_group_40[y1_entry], '^', color= color_dict['40%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

 #Group 2sigma
x_error = '206/204 corrected_2σ'
y_error = '208/204 corrected_2σ'
ax.errorbar(test_group_zero[x1_entry], test_group_zero[y1_entry],test_group_zero[x_error], test_group_zero[y_error],color = 'black')
ax.errorbar(test_group_20[x1_entry], test_group_20[y1_entry],test_group_20[x_error], test_group_20[y_error],color = 'black')
ax.errorbar(test_group_40[x1_entry], test_group_40[y1_entry],test_group_40[x_error], test_group_40[y_error],color = 'black')

 #Published value
x1_entry = '206/204'
y1_entry = '208/204'
ax.plot(pub_ratio[x1_entry], pub_ratio[y1_entry], 's', color= color_dict['pub'], markersize=18, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

#Formatting
ax.set(xlabel = '206/204', ylabel = '208/204',)
ax.yaxis.label.set_size(20)
ax.xaxis.label.set_size(20)



#207/206 vs 208/206 plot

x3_entry = '208/206 corrected'
y3_entry = '207/206 corrected'
ax = axs[1,0]
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)


custom_lines = ax.plot(test_zero[x3_entry], test_zero[y3_entry], 'o', color= color_dict['0%']) #0% reject
ax.plot(test_20[x3_entry], test_20[y3_entry], 'o', color= color_dict['20%']) #20% reject
ax.plot(test_40[x3_entry], test_40[y3_entry], 'o', color= color_dict['40%']) #40% reject

x1_entry = '208/206 corrected_mean'
y1_entry = '207/206 corrected_mean'
ax.plot(test_group_zero[x1_entry], test_group_zero[y1_entry], '^', color= color_dict['0%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
ax.plot(test_group_20[x1_entry], test_group_20[y1_entry], '^', color= color_dict['20%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
ax.plot(test_group_40[x1_entry], test_group_40[y1_entry], '^', color= color_dict['40%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

 #Group 2sigma
x_error = '208/206 corrected_2σ'
y_error = '207/206 corrected_2σ'
ax.errorbar(test_group_zero[x1_entry], test_group_zero[y1_entry],test_group_zero[x_error], test_group_zero[y_error],color = 'black')
ax.errorbar(test_group_20[x1_entry], test_group_20[y1_entry],test_group_20[x_error], test_group_20[y_error],color = 'black')
ax.errorbar(test_group_40[x1_entry], test_group_40[y1_entry],test_group_40[x_error], test_group_40[y_error],color = 'black')

 #Published value
x1_entry = '208/206'
y1_entry = '207/206'
ax.plot(pub_ratio[x1_entry], pub_ratio[y1_entry], 's', color= color_dict['pub'], markersize=18, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

#Formatting
ax.set(xlabel = '208/206', ylabel = '207/206',)
ax.yaxis.label.set_size(20)
ax.xaxis.label.set_size(20)


#206/238 vs 208/232 plot

x4_entry = '208/232 corrected'
y4_entry = '206/238 corrected'
ax = axs[1,1]
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)

ax.plot(test_zero[x4_entry], test_zero[y4_entry], 'o', color= color_dict['0%']) #0% reject
ax.plot(test_20[x4_entry], test_20[y4_entry], 'o', color= color_dict['20%']) #20% reject
ax.plot(test_40[x4_entry], test_40[y4_entry], 'o', color= color_dict['40%']) #40% reject

x1_entry = '208/232 corrected_mean'
y1_entry = '206/238 corrected_mean'
ax.plot(test_group_zero[x1_entry], test_group_zero[y1_entry], '^', color= color_dict['0%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
ax.plot(test_group_20[x1_entry], test_group_20[y1_entry], '^', color= color_dict['20%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
ax.plot(test_group_40[x1_entry], test_group_40[y1_entry], '^', color= color_dict['40%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

 #Group 2sigma
x_error = '208/232 corrected_2σ'
y_error = '206/238 corrected_2σ'
ax.errorbar(test_group_zero[x1_entry], test_group_zero[y1_entry],test_group_zero[x_error], test_group_zero[y_error],color = 'black')
ax.errorbar(test_group_20[x1_entry], test_group_20[y1_entry],test_group_20[x_error], test_group_20[y_error],color = 'black')
ax.errorbar(test_group_40[x1_entry], test_group_40[y1_entry],test_group_40[x_error], test_group_40[y_error],color = 'black')

 #Published value
x1_entry = '208/232'
y1_entry = '206/238'
ax.plot(pub_ratio[x1_entry], pub_ratio[y1_entry], 's', color= color_dict['pub'], markersize=18, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

#Formatting
ax.set(xlabel = '208/232', ylabel = '206/238',)
ax.yaxis.label.set_size(20)
ax.xaxis.label.set_size(20)

# Make a legend

patch_0 = mpatches.Patch(color= color_dict['0%'], label= '0% Rejection')
patch_20 = mpatches.Patch(color=color_dict['20%'], label= '20% Rejection')
patch_40 = mpatches.Patch(color=color_dict['40%'], label= '40% Rejection')
patch_pub = mpatches.Patch(color=color_dict['pub'], label= 'Published Data')
plt.legend(handles=[patch_0, patch_20, patch_40, patch_pub])







In [ ]:
plot_dict_grouped_dict

In [ ]:
def ratio_plot(group_namer, primary_std, choice = True, plot_dict = plot_dict, published_df = published_df):

    # Making groups of the standards for plotting
    worksheets = list(plot_dict.keys())
    group = None
    std_names = []
    start = [0]
    end = []
    for idx in range(len(worksheets)):
        if group == None:
            group = worksheets[idx].split('_')[0]
            std_names.append(group)
        #print(samples[idx])
        if worksheets[idx].split('_')[0] in group:
            pass
        else:
            end.append(idx)
            start.append(idx) 
            name = worksheets[idx].split('_')[0]
            group = name
            std_names.append(name)
    end.append(len(worksheets))

    #print('start:', start)
    #print('end:', end)
    #print('std names:', std_names)

    std_dict = {}

    for idx in range(len(std_names)):
        group_dict = {}
        std_group = worksheets[start[idx]:end[idx]]
        #print('std_group', std_group)
        group_dict['0%'] = std_group[0]
        group_dict['20%'] = std_group[2]
        group_dict['40%'] = std_group[4]

        group_dict['0% group'] = std_group[1]
        group_dict['20% group'] = std_group[3]
        group_dict['40% group'] = std_group[5]

        std_dict[std_names[idx]] = group_dict


    #Creating loop for all entries in plot_dict

    plot_dict_grouped_dict = {}

    for key in plot_dict:

        if 'full' in key:

        #Create groups for plotting

            samples = plot_dict[key].index.values.tolist()
            group = None
            group_names = []
            start = [0]
            end = []
            for idx in range(len(samples)):
                if group == None:
                    group = samples[idx].split(' 1')[0]
                    group_names.append(group)
                #print(samples[idx])
                if group in samples[idx]:
                    pass
                else:
                    end.append(idx)
                    start.append(idx) # + 1 was original
                    name = samples[idx].split(' 1')[0]
                    group = name
                    group_names.append(name)
            end.append(len(samples))

            #print('start:', start)
            #print('end:', end)
            #print('group names:', group_names)

            group_df_list = []
            group_df_dict = {}

            for idx in range(len(group_names)):
                group_dict = {}
                #print(idx)
                group_df = plot_dict[key].iloc[start[idx]:end[idx]]
                #print(group_df.index.values.tolist())

                group_df_list.append(group_df)
                group_df_dict[group_names[idx]] = group_df

            plot_dict_grouped_dict[key] = group_df_dict

            if 'group' in key:
                 plot_dict_grouped_dict[key] = plot_dict[key]


    #Data to be plotted


#     group_namer = 'Mudtank'
#     primary_std = 'ttn'

    primary_std_name = {
        'glass': "SRM NIST 610",
        'glass612': "SRM NIST 612",
        'glassBHVO': "BHVO-2G",
        'ttn': "MKED-1",
    }

    test_zero = plot_dict_grouped_dict[ std_dict[primary_std]['0%']][group_namer]
    test_20 = plot_dict_grouped_dict[ std_dict[primary_std]['20%']][group_namer]
    test_40 = plot_dict_grouped_dict[ std_dict[primary_std]['40%']][group_namer]

    test_group_zero = plot_dict[std_dict[primary_std]['0% group']].loc[group_namer]
    test_group_20 = plot_dict[std_dict[primary_std]['20% group']].loc[group_namer]
    test_group_40 = plot_dict[std_dict[primary_std]['40% group']].loc[group_namer]


    if (group_namer == 'Bancroft') or (group_namer =='Yates'):
        #print('debug')
        pub_ratio = published_df.loc[['OLT-1', 'OLT-2', 'TCB']]
    else:
        pub_ratio = published_df.loc[group_namer]


    color_dict = {
        '0%': 'red',
        '20%': 'blue',
        '40%': 'green',
        'pub': 'purple'
    }


    #Initialize plots
    fig, axs = plt.subplots(2, 2, sharex = False, figsize = (24, 12))
    fig.suptitle(group_namer + ', Primary Std: ' + primary_std_name[primary_std], fontsize=28)

    #207/204 vs 206/204 plot
        #Results for each line
    x1_entry = '206/204 corrected'
    y1_entry = '207/204 corrected'
    ax = axs[0,0]
    plt.setp(ax.get_xticklabels(), fontsize=16)
    plt.setp(ax.get_yticklabels(), fontsize=16)

    ax.plot(test_zero[x1_entry], test_zero[y1_entry], 'o', color= color_dict['0%']) #0% reject
    ax.plot(test_20[x1_entry], test_20[y1_entry], 'o', color= color_dict['20%']) #20% reject
    ax.plot(test_40[x1_entry], test_40[y1_entry], 'o', color= color_dict['40%']) #40% reject
        #Group averages
    x1_entry = '206/204 corrected_mean'
    y1_entry = '207/204 corrected_mean'
    ax.plot(test_group_zero[x1_entry], test_group_zero[y1_entry], '^', color= color_dict['0%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
    ax.plot(test_group_20[x1_entry], test_group_20[y1_entry], '^', color= color_dict['20%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
    ax.plot(test_group_40[x1_entry], test_group_40[y1_entry], '^', color= color_dict['40%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
        #Group 2sigma
    x_error = '206/204 corrected_2σ'
    y_error = '206/204 corrected_2σ'
    ax.errorbar(test_group_zero[x1_entry], test_group_zero[y1_entry],test_group_zero[x_error], test_group_zero[y_error],color = 'black')
    ax.errorbar(test_group_20[x1_entry], test_group_20[y1_entry],test_group_20[x_error], test_group_20[y_error],color = 'black')
    ax.errorbar(test_group_40[x1_entry], test_group_40[y1_entry],test_group_40[x_error], test_group_40[y_error],color = 'black')
        #Published value
    x1_entry = '206/204'
    y1_entry = '207/204'
    ax.plot(pub_ratio[x1_entry], pub_ratio[y1_entry], 's', color= color_dict['pub'], markersize=18, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

        #Formatting
    ax.set(xlabel = '206/204', ylabel = '207/204',)
    ax.yaxis.label.set_size(20)
    ax.xaxis.label.set_size(20)


    #208/204 vs 206/204 plot

    x2_entry = '206/204 corrected'
    y2_entry = '208/204 corrected'
    ax = axs[0,1]
    plt.setp(ax.get_xticklabels(), fontsize=16)
    plt.setp(ax.get_yticklabels(), fontsize=16)

    ax.plot(test_zero[x2_entry], test_zero[y2_entry], 'o', color= color_dict['0%']) #0% reject
    ax.plot(test_20[x2_entry], test_20[y2_entry], 'o', color= color_dict['20%']) #20% reject
    ax.plot(test_40[x2_entry], test_40[y2_entry], 'o', color= color_dict['40%']) #40% reject

    x1_entry = '206/204 corrected_mean'
    y1_entry = '208/204 corrected_mean'
    ax.plot(test_group_zero[x1_entry], test_group_zero[y1_entry], '^', color= color_dict['0%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
    ax.plot(test_group_20[x1_entry], test_group_20[y1_entry], '^', color= color_dict['20%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
    ax.plot(test_group_40[x1_entry], test_group_40[y1_entry], '^', color= color_dict['40%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

     #Group 2sigma
    x_error = '206/204 corrected_2σ'
    y_error = '208/204 corrected_2σ'
    ax.errorbar(test_group_zero[x1_entry], test_group_zero[y1_entry],test_group_zero[x_error], test_group_zero[y_error],color = 'black')
    ax.errorbar(test_group_20[x1_entry], test_group_20[y1_entry],test_group_20[x_error], test_group_20[y_error],color = 'black')
    ax.errorbar(test_group_40[x1_entry], test_group_40[y1_entry],test_group_40[x_error], test_group_40[y_error],color = 'black')

     #Published value
    x1_entry = '206/204'
    y1_entry = '208/204'
    ax.plot(pub_ratio[x1_entry], pub_ratio[y1_entry], 's', color= color_dict['pub'], markersize=18, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

    #Formatting
    ax.set(xlabel = '206/204', ylabel = '208/204',)
    ax.yaxis.label.set_size(20)
    ax.xaxis.label.set_size(20)



    #207/206 vs 208/206 plot

    x3_entry = '208/206 corrected'
    y3_entry = '207/206 corrected'
    ax = axs[1,0]
    plt.setp(ax.get_xticklabels(), fontsize=16)
    plt.setp(ax.get_yticklabels(), fontsize=16)


    custom_lines = ax.plot(test_zero[x3_entry], test_zero[y3_entry], 'o', color= color_dict['0%']) #0% reject
    ax.plot(test_20[x3_entry], test_20[y3_entry], 'o', color= color_dict['20%']) #20% reject
    ax.plot(test_40[x3_entry], test_40[y3_entry], 'o', color= color_dict['40%']) #40% reject

    x1_entry = '208/206 corrected_mean'
    y1_entry = '207/206 corrected_mean'
    ax.plot(test_group_zero[x1_entry], test_group_zero[y1_entry], '^', color= color_dict['0%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
    ax.plot(test_group_20[x1_entry], test_group_20[y1_entry], '^', color= color_dict['20%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
    ax.plot(test_group_40[x1_entry], test_group_40[y1_entry], '^', color= color_dict['40%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

     #Group 2sigma
    x_error = '208/206 corrected_2σ'
    y_error = '207/206 corrected_2σ'
    ax.errorbar(test_group_zero[x1_entry], test_group_zero[y1_entry],test_group_zero[x_error], test_group_zero[y_error],color = 'black')
    ax.errorbar(test_group_20[x1_entry], test_group_20[y1_entry],test_group_20[x_error], test_group_20[y_error],color = 'black')
    ax.errorbar(test_group_40[x1_entry], test_group_40[y1_entry],test_group_40[x_error], test_group_40[y_error],color = 'black')

     #Published value
    x1_entry = '208/206'
    y1_entry = '207/206'
    ax.plot(pub_ratio[x1_entry], pub_ratio[y1_entry], 's', color= color_dict['pub'], markersize=18, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

    #Formatting
    ax.set(xlabel = '208/206', ylabel = '207/206',)
    ax.yaxis.label.set_size(20)
    ax.xaxis.label.set_size(20)


    #206/238 vs 208/232 plot

    x4_entry = '208/232 corrected'
    y4_entry = '206/238 corrected'
    ax = axs[1,1]
    plt.setp(ax.get_xticklabels(), fontsize=16)
    plt.setp(ax.get_yticklabels(), fontsize=16)

    ax.plot(test_zero[x4_entry], test_zero[y4_entry], 'o', color= color_dict['0%']) #0% reject
    ax.plot(test_20[x4_entry], test_20[y4_entry], 'o', color= color_dict['20%']) #20% reject
    ax.plot(test_40[x4_entry], test_40[y4_entry], 'o', color= color_dict['40%']) #40% reject

    x1_entry = '208/232 corrected_mean'
    y1_entry = '206/238 corrected_mean'
    ax.plot(test_group_zero[x1_entry], test_group_zero[y1_entry], '^', color= color_dict['0%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
    ax.plot(test_group_20[x1_entry], test_group_20[y1_entry], '^', color= color_dict['20%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject
    ax.plot(test_group_40[x1_entry], test_group_40[y1_entry], '^', color= color_dict['40%'], markersize=14, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

     #Group 2sigma
    x_error = '208/232 corrected_2σ'
    y_error = '206/238 corrected_2σ'
    ax.errorbar(test_group_zero[x1_entry], test_group_zero[y1_entry],test_group_zero[x_error], test_group_zero[y_error],color = 'black')
    ax.errorbar(test_group_20[x1_entry], test_group_20[y1_entry],test_group_20[x_error], test_group_20[y_error],color = 'black')
    ax.errorbar(test_group_40[x1_entry], test_group_40[y1_entry],test_group_40[x_error], test_group_40[y_error],color = 'black')

     #Published value
    x1_entry = '208/232'
    y1_entry = '206/238'
    ax.plot(pub_ratio[x1_entry], pub_ratio[y1_entry], 's', color= color_dict['pub'], markersize=18, markeredgewidth=1.5, markeredgecolor= 'black') #0% reject

    #Formatting
    ax.set(xlabel = '208/232', ylabel = '206/238',)
    ax.yaxis.label.set_size(20)
    ax.xaxis.label.set_size(20)

    # Make a legend

    patch_0 = mpatches.Patch(color= color_dict['0%'], label= '0% Rejection')
    patch_20 = mpatches.Patch(color=color_dict['20%'], label= '20% Rejection')
    patch_40 = mpatches.Patch(color=color_dict['40%'], label= '40% Rejection')
    patch_pub = mpatches.Patch(color=color_dict['pub'], label= 'Published Data')
    plt.legend(handles=[patch_0, patch_20, patch_40, patch_pub])


    MYDIR = ("Ratio_Figures")
    CHECK_FOLDER = os.path.isdir(MYDIR)

    # If folder doesn't exist, then create it.
    if not CHECK_FOLDER:
        os.makedirs(MYDIR)
        #print("created folder : ", MYDIR)
    
    new_string = group_namer + '_PrimaryStd_' + primary_std_name[primary_std]
    
    filename = os.path.join(MYDIR, new_string + '.pdf')
    plt.savefig(filename)
    print('Plot for ', new_string, " is complete.")
    
    if choice == False:
        plt.close()





In [ ]:
def ratio_report(plot_dict = plot_dict, intro_filename = 'fake.pdf', intro = False, output_name = 'ratio_output.pdf'):
    
    MYDIR = ("Ratio_Figures")
    mergedObject = PdfFileMerger()
    
    if intro:
        mergedObject.append(PdfFileReader(intro_filename, 'rb'))
        print(f'Succesfully incorporated {intro_filename} into PDF.')
    
    
    #Actual loop of plotting
    
    # Making groups of the standards for plotting
    worksheets = list(plot_dict.keys())
    group = None
    std_names = []
    start = [0]
    end = []
    for idx in range(len(worksheets)):
        if group == None:
            group = worksheets[idx].split('_')[0]
            std_names.append(group)
        #print(samples[idx])
        if worksheets[idx].split('_')[0] in group:
            pass
        else:
            end.append(idx)
            start.append(idx) 
            name = worksheets[idx].split('_')[0]
            group = name
            std_names.append(name)
    end.append(len(worksheets))

    #print('start:', start)
    #print('end:', end)
    #print('std names:', std_names)

    group_list = None

    for key in plot_dict:

        if 'full' in key:

        #Create groups for plotting

            samples = plot_dict[key].index.values.tolist()
            group = None
            group_names = []
            start = [0]
            end = []
            for idx in range(len(samples)):
                if group == None:
                    group = samples[idx].split(' 1')[0]
                    group_names.append(group)
                #print(samples[idx])
                if group in samples[idx]:
                    pass
                else:
                    end.append(idx)
                    start.append(idx) # + 1 was original
                    name = samples[idx].split(' 1')[0]
                    group = name
                    group_names.append(name)
            end.append(len(samples))

            #print('start:', start)
            #print('end:', end)
            #print('group names:', group_names)
            group_list = group_names

    #print('group names:', group_list)

    primary_std_name = {
        'glass': "SRM NIST 610",
        'glass612': "SRM NIST 612",
        'glassBHVO': "BHVO-2G",
        'ttn': "MKED-1",
    }
    
    
    for std in std_names:
        #print(std)
        for group in group_list:
            #print(group)
            ratio_plot(group, std, False)
            new_string = group + '_PrimaryStd_' + primary_std_name[std]
    
            filename = os.path.join(MYDIR, new_string + '.pdf')
            mergedObject.append(PdfFileReader(filename, 'rb'))
    
    if '.pdf' in output_name:
        pass
    else:
        output_name = output_name + '.pdf'
    
     
    mergedObject.write(output_name)

    print(f'PDF file named: {output_name} is complete.')   

In [ ]:
ratio_report()

In [ ]:
# excel_name = '06May_ttnSS2_NP2_processed.xlsx'
# files_ranked_toEXCEL(tester, excel_name)

In [ ]:
#filename = '0416_glass_SS_NPII_baseline.xlsx'
#calc_dict = calc_CPS(read_np2_timeseries(filename))

In [ ]:
# filename = '06May_ttnSS2_NP2_baseline_corrected.xlsx'
# SS_dict = calc_CPS(read_np2_timeseries(filename))
# excel_name = str(filename.split('.')[0]) + '_processed.xlsx'

# files_process_toEXCEL(SS_dict, excel_name)

# U_Pb_report(SS_dict, 'SS2_6May.pdf', True, 'Splitstream_06May2021_results')

In [ ]:
# filename = '0416_ttn_SS_NP2_data1.xlsx'
# ttn_dict = calc_CPS(read_np2_timeseries(filename))

# U_Pb_report(ttn_dict, 'titaniteTE.pdf',True, 'Titanite_splitstream_results')

In [ ]:
#U_Pb_report(calc_dict, 'glassTE.pdf',True, 'Glass_splitstream_results')